# **Introduction to Computer Theory, 2nd. Edition by Daniel Cohen**

CSCI-549 East Texas A&M University

Lon Cherryholmes, Sr.

In [ ]:
!pip install SNOBOL4python==0.4.5
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz
!pip install rustworkx

In [ ]:
import re
import math
import graphviz
import itertools
import rustworkx as rx
from PIL import Image
from io import BytesIO
from graphviz import Digraph
from pygraphviz import AGraph
from itertools import product
from rustworkx import vf2_mapping
from pprint import pformat, pprint
from IPython.display import display, Latex, Markdown, SVG
## Thirty one (31) flavors of patterns to choose from ...
from SNOBOL4python import ε, σ, π, λ, Λ, ζ, θ, Θ, φ, Φ, α, ω
from SNOBOL4python import ABORT, ANY, ARB, ARBNO, BAL, BREAK, BREAKX, FAIL
from SNOBOL4python import FENCE, LEN, MARB, MARBNO, NOTANY, POS, REM, RPOS
from SNOBOL4python import RTAB, SPAN, SUCCESS, TAB
# Miscellaneous
from SNOBOL4python import GLOBALS, TRACE, PATTERN, STRING
from SNOBOL4python import ALPHABET, DIGITS, UCASE, LCASE, NULL
from SNOBOL4python import nPush, nInc, nPop, Shift, Reduce, Pop
# Instantiate the global variable space
GLOBALS(globals())
TRACE(40)
FAs = {}
TGs = {}
GTGs = {}
NFAs = {}
Moores = {}
Mealys = {}

# **Definitions**

## length function

We define the function **length** of a string to be the number of letters in the string.

## reverse function

Let us introduce the function **reverse**. If *a* is a word in some language $L$, then reverse(*a*) is the same string of letters spelled backward, called the reverse of *a*, even if this backward string is not a word in $L$.

## Kleene closure

Given an alphabet Σ, we wish to define a language in which any string of letters from Σ is a word, even the null string. This language we shall call the **closure** of the alphabet. It is denoted by writing a star (an asterisk) after the name of the alphabet as a superscript: $$Σ^{*}$$ This notation is sometimes known as the **Kleene star** after the logician who was one of the founders of this subject.

If $S$ is a set of words, then by $S^{*}$ we mean the set of all finite strings formed by concatenating words from $S$, where any word may be used as often as we like, and where the null string is also included.

## regular expression

The symbols that appear in regular expressions are the letters of the alphabet $Σ$, the symbol for the null string $\mathbf{Λ}$, parenthesis, the star operator, and the plus sign.

The set of **regular expressions** is defined by the following rules:

Rule 1: Every letter of Σ can be made into a regular expression by writing it in boldface. **Λ** itself is a regular expression.

Rule 2: If $\mathbf{r_1}$ and $\mathbf{r_2}$ are regular expressions, then so are:
* (i) $(\mathbf{r_1})$
* (ii) $\mathbf{r_1r_2}$
* (iii) $\mathbf{r_1}+\mathbf{r_2}$
* (iv) $\mathbf{r_1}^{*}$

Rule 3: Nothing else is a regular expression.

## language associated (with regular expression)

The following rules define the **language associated** with any regular expression:

* Rule 1: The language associated with the regular expression that is just a single letter is that one-letter word alone, and the language associated with $Λ$ is just { **Λ** }, a one-word language.

* Rule 2: If $\mathbf{r_1}$ is a regular expression associated with the language $L_1$ and $\mathbf{r_2}$ is a regular expression associated with the language $L_2,$ then:
  - (i) The regular expression $(\mathbf{r_1})(\mathbf{r_2})$ is associated with the product $L_1L_2$ that is the the language $L_1$ times $L_2$: $$\text{language(}\mathbf{r_1}\mathbf{r_2}\text{)}=L_1L_2$$
  - (ii) The regular expression $\mathbf{r_1}+\mathbf{r_2}$ is associated with the language formed by the union of the sets $L_1$ and $L_2$: $$\text{language(}\mathbf{r_1}+\mathbf{r_2}\text{)}=L_1+L_2$$
  - (iii) The language associated with the regular expression ($\mathbf{r_1}^{*}$) is $L_1^{*}$, the Kleene closure of the set $L_1$ as a set of words: $$\text{language(}\mathbf{r_1}^{*}\text{)}=L_1^{*}$$

## product set

If $S$ and $T$ are sets of strings of letters (whether they are finite or infinite sets), we define the **product set** of strings of letters to be:

$$
ST= \{ \text{all combinations of a string from S concatenated with a string from T in that order } \}
$$

## finite automaton

A **finite automaton**, abbreviated **FA**, is a collection of three things:

1. A finite set of states, one of which is designated as the initial state, called the **start state**, and some (maybe none) of which are designated as **final states**.
2. An **alphabet** $Σ$ of possible input letters.
3. A finite set of **transitions** that tell for each state and for each letter of the input alphabet which state to go to next.

The plural of automaton, **finite automata**, is abbreviated **FAs**.

The abstract definition of an FA is then:

1. A finite set of states $Q = \{ q_0\; q_1\; q_2\; ... \}$ of which $q_0$ is the start state
2. A subset of $Q$ called the final states.
3. An alphabet $Σ = \{ x_1\; x_2\; x_3\; ... \}$.
4. A transition function $δ$ associating each pair of state and letter with a state: $δ(q_{i}, x_{j})=x_k$

## transition graph

A **transition graph**, abbreviated **TG**, is a collection of three things:

1. A finite set of states, at least one of which is designated as the **start state** (*-*) and some (maybe none) of which are designated as **final states** (*+*).
2. An **alphabet** $Σ$ of possible input letters from which input strings are formed.
3. A finite set of **transitions** (edge labels) that show how to go from some states to some others, based on reading specified substrings of input letters (possibly even the null string **Λ**).

## generalized transition graph

A **generalized transition graph**, abbreviated **GTG**, is a collection of three things:

1. A finite set of states, of which at least one is a **start state** and some (maybe none) are **final states**.
2. An **alphabet** $Σ$ of input letters.
3. Directed edges connecting some pairs of states, each labeled with a regular expression.

## nondeterministic finite automata

A **nondeterministic finite automata** is a TG with a unique **start state** with the property that each of its edge labels is a single alphabet letter. It is given the acronym NFA. Sometimes, to distinguish them from NFAs, the regular deterministic finite automata are referred to as DFAs.

## Moore machine

A Moore machine is a collection of five things:

1. A finite set of states $q_0, q_1, q_2, ...,$ where $q_0$ is designated as the start state.
2. An alphabet of letters for forming the input string $$Σ = \{ a\, b\, c\, ... \}$$
3. An alphabet of possible output characters $$Γ = \{ x\, y\, z\, ... \}$$
4. A transition table that shows for *each* state and *each* input letter what state is reached next.
5. An output table that shows what character from Γ is printed by each state as it is entered.

## Mealy machine

A Mealy machine is a collection of four things:

1. A finite set of states $q_0, q_1, q_2, ...,$ where $q_0$ is designated as the start state.
2. An alphabet of letters $Σ = \{ a\, b\, c\, ... \}$ for forming input strings.
3. An alphabet of possible output characters $Γ = \{ x\, y\, z\, ... \}$.
4. A pictorial representation with states represented by small circles and directed edges indicating transitions between states. Each edge is labeled with a compound symbol of the form *i/o*, where *i* is the input symbol and *o* is the output character.*Every* state must have exactly one outgoing edge for *each* possible input letter. The edge we travel is determined by the input letter *i*. While traveling on the edge, we must print the output character *o*.

# **Theorems**

## Theorem 1:

For any set $S$ of strings we have $S^{*}=S^{**}$.

## Theorem 2:

An arithmetic expression cannot contain the character *\$*.

## Theorem 3:

No arithmetic expression can begin or end with the symbol */*.

## Theorem 4:

No arithmetic expression can contain the substring *//*.

## Theorem 5:

If $L$ is a finite language (a language with only finitely many words), then $L$ can be defined by a regular expression. In other words, all finite languages are regular.

## Theorem 6:

Any language that can be defined by
* regular expression, or
* finite automaton, or
* transition graph

can be defined by all three methods.

## Theorem 7:

For every NFA, there is some FA that accepts exactly the same language.

## Theorem 8:

If $Mo$ is a Moore machine, then there is a Mealy machine $Me$ that is equivalent to it.

## Theorem 9:

For every Mealy machine $Me$, there is a Moore machine  $Mo$ that is equivalent to it.

## Theorem 10:

If $L_1$ and $L_2$ are regular expressions, then $L_1+L_2$, $L_1L_2$, and $L_1^{*}$ are also regular expressions.

## Theorem 11:

If $L$ is a regular language, then $L'$ is also a regular language. In other words, the set of regular languages is closed under complementation.

## Theorem 12:

If $L_1$ and $L_2$ are regular languages, then $L_1 \cap L_2$ is also a regular language. In other words, the set of regular languages is closed under intersection.

## Theorem 13:

Let $L$ be any regular language that has infintely many words. Then there exists some three strings *x*, *y*, and *z* (where *y* is not the null string) such that all the strings of the form $$xy^nz\; \text{for n}=1\,2\,3\,...$$ are words in $L$.

## Theorem 14:

Let $L$ be an infinite language accepted by a finite automaton with $N$ states. Then for all words $w$ in $L$ that have more than $N$ letters, there are strings $x$, $y$, and $z$, where $y$ is not null and length($x$) + length($y$) does not exceed $N$ such that $$w=xyz$$ and all strings of the form $$xy^nz\; \text{(for } n=1\, 2\, 3\, ...\text{)}$$ are in $L$.

## Theorem 15:

Given a language $L$, we shall say that any two strings $x$ and $y$ are in the same class if for all possible strings $z$ either both $xz$ and $yz$ are in $L$ or both are not.

1. The language $L$ divides the set of all possible strings into separate (mutually exclusive) classes.
2. If $L$ is regular, the number of classes $L$ creates is finite.
3. If the number of classes $L$ creates is finite, then $L$ is regular.

## Theorem 16:

If $R$ is a regular language and $Q$ is any language whatsoever, then the language $$P=\text{Pref}(Q\; \text{in}\; R)$$ is regular.

# **Algorithms**

## Construct regular expression from TG

* Step 1: Create a unique, unenterable minus state and a unique, unleaveable plus state.
* Step 2: One by one, in any order, bypass and eliminate all the non *-* or *+* states in the TG. A state is bypassed by connecting each incoming edge with each outgoing edge. The label of each resultant edge is the concatenation of the label on the incoming edge with the label on the loop edge if there is one and the label on the outgoing edge.
* Step 3: When two states are joined by more than one edge going in the same direction, unify them by adding their labels.
* Step 4: Finally, when all that is left is one edge from *-* to *+*, the label on that edge is a regular expression that generates the same language as was recognized by the original machine.


## Construct union of finite automata

p. 113

## Construct product of finite automata

p. 121

## Construct closure of finite automaton

p. 129

# **Chapter 2: Languages**

## Functions

In [ ]:
from itertools import product
from pprint import pprint

def kleene_star(S):
    result = [{} for _ in range(8)]
    for repeat in range(8):
        for combo in product(S, repeat=repeat):
            combo = "".join(combo)
            length = len(combo)
            if length < len(result):
                if combo not in result[length]:
                    result[length][combo] = 1
                else: result[length][combo] += 1
    for length, words in enumerate(result):
        if words is not None:
            print(length, len(words), words)

## Problem 2.1:

---
Consider the language $S^{*}$, where $S =$ {*a b*}.
1. How many words does the language have of length 2?
2. of length 3?
3. of length *n*?

**Answer 2.1:**
1. $4$
2. $8$
3. $2^{n}$

In [ ]:
# Observe the powers of two sequence: 1, 2, 4, 8, 16, 32, 64, ...
S = {"a", "b"}
kleene_star(S)

## Problem 2.2:

---
Consider the language $S^{*}$, where $S =$ {*aa b*}.
1. How many words does the language have of length 4?
2. of length 5?
3. of length 6?
4. What can be said in general?

**Answer 2.2:**
1. $5$
2. $8$
3. $13$
4. *See below.**

In [ ]:
# Observe the Fibonacci sequence: 1, 1, 2, 3, 5, 8, 13, ...
S = {"aa", "b"}
kleene_star(S)

## Problem 2.3:

---
Consider the language $S^{*}$, where $S =$ {*ab ba*}.
1. Write out all the words in $S^{*}$ that have seven or fewer letters.
2. Can any word in this language contain the substrings *aaa* or *bbb*?
3. What is the smallest word *not* in this language?

**Answer 2.3:**

1. *See below.*
2. No. $S$ does not contain *aa* or *bb*. Any substring has at most two consecutive *a*'s or *b*'s.
3. Either *a* or *b*.

In [ ]:
# Answer 2.3.1
S = {"ab", "ba"}
kleene_star(S)

## Problem 2.4:

---
Consider the language $S^{*}$, where $S =$ {*a ab ba*}.
1. Is the string (*abbba*) a word in this language?
2. Write out all the words in this language with six or fewer letters.
3. What is another way in which to describe the words in this language? Be careful, this is not simply the language of all words without bbb.

**Answer 2.4:**

1. No. Neither *b* nor *bb* are in $S$. At most two consecutive *b*'s.
2. *See below.*
3. All the strings of *a*'s and *b*'s where each *b* is adjacent to a unique *a*.

In [ ]:
# Answer 2.4.2
S = {"a", "ab", "ba"}
kleene_star(S)

## Problem 2.5:

---
Consider the language $S^{*}$, where $S =$ {*aa aba baa*}.
1. Show that the words *aabaa*, *baaabaaa*, and *baaaaababaaaa* are all in this language.
2. Can any word in this language be interpreted as a string of elements from $S$ in two different ways?
3. Can any word in this language have an odd total number of *a*'s?

**Answer 2.5:**
1. *aabaa* = *aa baa* \
   *baaabaaa* = *baa aba aa* \
   *baaaaababaaaa* = *baa aa aba baa aa*
2. No. None of the words in $S$ is a prefix of another word in $S$. Neither *a*, *ab*, nor *ba* are elements of $S$.
3. No. Each element has two *a*'s and any composite string will have only multiples.

## Problem 2.6:

---
Consider the language $S^{*}$, where $S =$ {*xx xxx*}.
In how many ways can $x^{19}$ be written as the product of words in $S$?
This means: How many different factorizations are there of $x^{19}$ into *xx* and *xxx*?

**Answer 2.6:**

$\text{Let } a = \#(\text{copies of } xx), \quad b = \#(\text{copies of } xxx)$.

The length condition is $2a + 3b = 19, \quad a,b \in \mathbb{Z}_{\ge 0}$.

For each feasible pair $(a,b)$, the number of distinct orderings is $\binom{a+b}{a}$.

Solving $2a + 3b = 19$:

$$
\begin{aligned}
b &= 1 \quad\Rightarrow\quad a = 8, &\binom{9}{8} &= 9, \\
b &= 3 \quad\Rightarrow\quad a = 5, &\binom{8}{5} &= 56, \\
b &= 5 \quad\Rightarrow\quad a = 2, &\binom{7}{2} &= 21.
\end{aligned}
$$

Summing over all solutions: $9 + 56 + 21 = 86$.

$$
\boxed{\text{There are 86 distinct factorizations.}}
$$

## Problem 2.7:

---
Consider the language PALINDROME over the alphabet {*a b*}.

$\textbf{Let}$ $\text{PALINDROME} = \{\, w \in \{a,b\}^* \mid w = w^R \,\}$, where $w^R$ denotes the reverse of $w$.

* (i) Prove that if $x$ is in PALINDROME, then so is $x^{n}$ for any *n*.
* (ii) Prove that if $y^{3}$ is in PALINDROME, then so is $y$.
* (iii) Prove that if $z^{n}$ is in PALINDROME for some $n$ (greater than 0), then $z$ itself is also.
* (iv) Prove that PALINDROME has as many words of length 4 as it does of length 3.
* (v) Prove that PALINDROME has as many words of length $2n$ as it has of length $2n - 1$. How many words is that?

---
**Answer 2.7(i):**

$\textbf{Claim.}$ If $x \in \text{PALINDROME}$, then $x^n \in \text{PALINDROME}$ for any integer $n \ge 0$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $x \in \text{PALINDROME}$, we have $x = x^R$. Thus, $(x^n)^R = (x^R)^n = x^n$.

---
**Answer 2.7(ii):**

$\textbf{Claim.}$ If $y^3 \in \text{PALINDROME}$, then $y \in \text{PALINDROME}$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $y^3 \in \text{PALINDROME}$, we have $yyy = (yyy)^R$, $(yyy)^R = y^Ry^Ry^R$. Given they are one-to-one and corresponding, therefore $y = y^R$.

---
**Answer 2.7(iii):**

$\textbf{Claim.}$ If $y^3 \in \text{PALINDROME}$, then $y \in \text{PALINDROME}$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $z^n \in \text{PALINDROME}$, we have $z^n = (z^n)^R$, $(z^n)^R = (z^R)^n$. Given they are one-to-one and corresponding, therefore $z = z^R$.

**Answer 2.7(iv):**

For alphabet {*a b*}, a palindrome of length $n$ is determined by its first ${\lceil n/2 \rceil}$ positions, each with 2 choices. Thus the number of palindromes of length $n$ is $2^{\lceil n/2 \rceil}$. For $n=3$ the length is $2$ and for $n=4$ the length is $2$, hence the same number of palindromes, which is $4$.

---
**Answer 2.7(v):**

For alphabet {*a b*}, a palindrome of length $n$ is determined by its first ${\lceil n/2 \rceil}$ positions, each with 2 choices. Thus the number of palindromes of length $n$ is $2^{\lceil n/2 \rceil}$. For $n=2N$ the length is ${\lceil 2N/2 \rceil}$ and for $n=2N-1$ the length is ${\lceil (2N-1)/2 \rceil}$, but $2N/2=(2N-1)/2=N$ hence the same number of palindromes, which is $2^{\lceil N \rceil}$.

## Problem 2.8:

---
Show that if the concatenation of two words (neither $Λ$) in PALINDROME is also a word in PALINDROME, then both words are powers of some other word; that is, if $x$ and $y$ and $xy$ are all in PALINDROME, then there is a word $z$ such that $x = z^{p}$  and $y = z^{q}$ for some integers $p$ and $q$ (maybe $p$ or $q = 1$).

* $x = x^R$
* $y = y^R$
* $(xy) = (xy)^R = (y^Rx^R) = (yx)$
  
so $x$ and $y$ commute.

We prove by induction on $|x|+|y|$ that if nonempty words $x,y$ commute, then there exists $z$ with $x=z^m$ and $y=z^n$ for some $m,n\ge 1$.

If $|x|=|y|$, then $xy=yx$ implies $x=y$, and taking $z=x$ gives the claim. Otherwise, without loss of generality $|x|>|y|$. Since $xy=yx$, the word $y$ is a prefix of $x$; write $x=yt$ for some (possibly empty) $t$. Substituting into $xy=yx$ yields

$(yt)y = y(yt) \;\;\Rightarrow\;\; yty = yyt \;\;\Rightarrow\;\; ty=yt$,

where we used left-cancellation in the free monoid. If $t=\varepsilon$, then $x=y$ and we are done as before. If $t\neq\varepsilon$, then $t$ and $y$ are shorter nonempty words that commute, and $|t|+|y|<|x|+|y|$. By the induction hypothesis, there exists a word $z$ and integers $p,q\ge 1$ such that $t=z^p$ and $y=z^q$. Hence

$x=yt = z^q z^p = z^{p+q} \quad\text{and}\quad y=z^q$,

so taking $m=p+q$ and $n=q$ completes the proof.

Thus, from $x,y,xy\in$ PALINDROME with $x,y\neq\varepsilon$, we conclude that $x$ and $y$ are powers of a common word.

## Problem 2.9:

---
* (i) Let $S$ = {*ab bb*} and let $T$ = {*ab bb bbbb*}. Show that $S^{*} = T^{*}$.
* (ii) Let $S$ = {*ab bb*} and let $T$ = {*ab bb bbb*}. Show that $S^{*} \neq T^{*}$, but that $S^{*} \subset T^{*}$.
* (iii) What principle does this illustrate?

Proof that $S^{*} = T^{*}$

Let $S =$ {*ab bb*}, $T =$ {*ab bb bbbb*}.

By definition of the Kleene star:

$
A^{0} = \{\varepsilon\},
A^{n} = \{x_{1}x_{2}\cdots x_{n} \mid x_{i} \in A \ \text{for all } i\},
A^{*} = \bigcup_{n \ge 0} A^{n}.
$

1. $S^{*} \subseteq T^{*}$

Let $w \in S^{*}$. Then $w \in S^{n}$ for some $n \ge 0$, so $w = s_{1}s_{2}\cdots s_{n}
\quad\text{with each } s_{i} \in S$.

Since $S \subseteq T$, each $s_{i} \in T$, hence $w \in T^{n} \subseteq T^{*}$.

2. $T^{*} \subseteq S^{*}$

Let $w \in T^{*}$. Then $w \in T^{k}$ for some $k \ge 0$, so $w = x_{1}x_{2}\cdots x_{k} \quad\text{with each } x_{i} \in T = \{ab,\, bb,\, bbbb\}$.

We can rewrite each \(x_{i}\) as a sequence of elements from $S$:
- If $x_{i} = ab$, replace it by $(ab)$.
- If $x_{i} = bb$, replace it by $(bb)$.
- If $x_{i} = bbbb$, note that $bbbb = (bb)(bb)$, so replace it by ($bb$ $bb$).

Concatenating all these replacements yields $w = y_{1}y_{2}\cdots y_{m}\quad\text{with each } y_{j} \in S$.

Thus $w \in S^{m} \subseteq S^{*}$.

3. Since $S^{*} \subseteq T^{*}$ and $T^{*} \subseteq S^{*}$, we have $S^{*} = T^{*}$.

## Problem 2.10:

---
How does the situation in Problem 9 change if we replace the operator $^{*}$ with the operator $^{+}$ as defined in this chapter? Note the language $S^{+}$ means the same same $S^{*}$, but does not allow the "concatenation of no words" of $S$.

**Answer 2.10:**
No changes in the the equalities and inclusions.

## Problem 2.11:

---
Prove that for all sets $S$,
* (i) $(S^{+})^{*} = (S^{*})^{*}$
* (ii) $(S^{+})^{+} = S^{+}$
* (iii) Is $(S^{*})^{+} = (S^{+})^{*}$ for all sets $S$?

**Answer 2.11:**

* (i) $(S^+)^*$ includes $Λ$ even if $S$ does not, so $(S^+)^*=S^*$. $S^*=S^**$ by Theorem 1.
* (ii) There can be no factor in $S$ that is not in $S^+$, $(S^+)^+ \subseteq S^+$. In general any set is contained in its positive closure $S^+ \subseteq (S^+)^+$. Therefore, $(S^+)^+ = S^+$.
* (iii) Yes.

## Problem 2.12:

---
Let $S$ = {*a bb bab abaab*}.
1. Is *abbabaabab* in $S^{*}$?
2. Is *abaabbabbaabb*?
3. Does any word in $S^{*}$ have an odd total number of *b*'s?

**Answer 2.12:**

1. No.
2. No.
3. No.

## Problem 2.13:

---
Suppose that for some language $L$ we can always concatenate two words in $L$ and get another word in $L$, if and only if the words are not the same. That is, for any words $w_{1}$ and $w_{2}$ in L where $w_{1} \neq w_{2}$, the word $w_{1}$$w_{2}$ is in $L$ but the word $w_{1}$$w_{1}$ is not in $L$. Prove that this cannot happen.

**Answer 2.13:** This is the same as saying that the language L would allow all concatenations that did not produce squares. First observe that Λ = ΛΛ, so Λ cannot be in the language. Consider $w_1 \neq w_2 $ and $ w_1w_2 \in L$. Let $w_3 = w_1w_2$, since $Λ \notin L$, $w_3 \neq w_2$, so $w_4 = w_2w_3 \in L$ where $w_4 \neq w_1$, finally let $w_5 = w_1w_4 \in L$. However, $w5 = w_1w_2w_1w_2 = w_3w_3$ which is square, so $w_5 \notin L$.

## Problem 2.14:

---
Let us define $(S^{**})^{*} = S^{***}$.

1. Is this set bigger than $S^{*}$?
2. Is it bigger that $S$?

**Answer 2.14:**

1. No. $(S^{**})^{*} = (S^{*})^{*} = S^{*}$ by Theorem 1.
2. It is often bigger than $S$.

## Problem 2.15:

---
Let $w$ be a string of letters and let the language $T$ be defined by adding $w$ to the language $S$. Suppose further that $T^{*} = S^{*}$.
* (i) Is it necessarily true that $w \in S$?
* (ii) Is it necessarily true that $w \in S^{*}$?

**Answer 2.15:**
* (i) no
* (ii) yes, $T = S + \{w\} \implies w \in T \implies w \in T^*$ and $T^* = S^* \implies w \in S^*$.

## Problem 2.16:

---
Give an example of a set $S$ such that the language $S^{*}$ has more six-letter words than eight-letter words. Does there exists an $S^{*}$ such that it has more six-letter words that twelve-letter words?

**Answer 2.16:**
Let $S = $ {*aaa*}, $S^*$ has one six-letter word and no seven-letter words and no eight-letter words. However, it is impossible for $S^*$ for $S$ to contain more six-letter words that twelve-letter words, because for every six-letter word $w$ there is a twelve-letter word $ww$ in $S^*$.

## Problem 2.17:

---
* (i) Consider the language $S^{*}$, where $S$ = {*aa ab ba bb*}. Give another description of this language.
* (ii) Give an example of a set $S$ such that $S^{*}$ *only* contains all possible strings of *a*'s and *b*'s that have length divisible by 3.
* (iii) Let S be all strings of *a*'s and *b*'s with odd length. What is $S^{*}$?

**Answer 2.17:**
- (i) All words over $Σ =$ {*a b*} of even length.
- (ii) $S =$ {*aaa aab aba abb baa bab bba bbb*}
- (iii) All strings of *a*'s and *b*'s except Λ

## Problem 2.18:

---
* (i) If $S =$ {*a b*} and $T^{*} = S^{*}$, prove that $T$ must contain $S$.
* (ii) Find another pair of sets $S$ and $T$ such that if $T^{*} = S^{*}$, then $S \subset T$.

**Answer 2.18:**
- (i) $S^*$ and $T^*$ both represent the set of all strings of *a*'s *b*'s. Therefore $T$ must include at least the words *a* and *b*, which is the set $S$.
- (ii) $S =$ {*a bb*}, $T =$ {*a aa bb*}.

## Problem 2.19:

---
One student suggested the following algorithm to test a string of *a*'s and *b*'s to see if it is a word in $S^{*}$, where $S =$ {*aa ba aba abaab*}.
* Step 1, cross off the longest set of characters from the front of the string that is a word in $S$.
* Step 2, repeat step 1 until it is no longer possible. If what remains is the string $Λ$, the original string was a word in $S^{*}$. If what remains is not $Λ$ (this means that some letters are left, but we cannot find a word in $S$ at the beginning), the original string was not a word in $S^{*}$. Find a string that disproves this algorithm.

**Answer 2.19:**
The word *abaaba* disproves the algorithm.

## Problem 2.20:

---
A language $L_1$ is smaller than another language $L_2$ if $L_1 \subset L_2$ and $L_1 \neq L_2$. Let $T$ be any language closed under concatenation; that is, if $t_1 \in T$ and $t_2 \in T$, then $t_1t_2$ is also an element of $T$. Show that if $T$ contains $S$ but $T \neq S^{*}$, then $S^{*}$ is smaller that $T$. We can summarize this by saying that $S^{*}$ is the smallest closed language containing $S$.

**Answer 2.20:**
Since $T$ is closed and $S \subset T$, any factors in $S$ concatentated together two at a time will be a word in $T$. Likewise concatentating factors in $S$ any number of times produces a word in $T$. That is any word in $S^*$ is also in $T$. However we are given that $T \neq S^*$ so $T$ contains some words not in $S^*$. We can conclude that $S^*$ is a proper subset of $T$, in other words $S^*$ is smaller that $T$, and in symbols $S^* \subset T$.

# **Chapter 3: Recursive Definitions**

## **Functions**

In [ ]:
def derivations(n):
    if n == 1:
        return ["1"]
    results = []
    for a in range(1, n):
        b = n - a
        if a > b:
            break
        for left in derivations(a):
            for right in derivations(b):
                if a == b:
                    if left > right:
                        continue
                results.append(f"{a}[{left}] + {b}[{right}]")
    return results

n = 7
seq = derivations(n)
print(f"Derivations for {n} (total {len(seq)} ways):")
for i, d in enumerate(seq, 1):
    print(f"{i}. {d}")

In [ ]:
from functools import lru_cache
@lru_cache(None)
def count_proofs(n):
    if n == 1:
        return 1
    total = 0
    for a in range(1, n):
        b = n - a
        if a > b:
            break
        if a == b:
            total += (count_proofs(a) * (count_proofs(a) + 1)) // 2
        else:
            total += count_proofs(a) * count_proofs(b)
    return total

for n in range(1, 16):
    print(f"{n}: {count_proofs(n)} proofs")

---
## **Problem archetypes**

1. Write a recursive definition given a statement of the language
2. Prove a recursive definition correctly specifies the stated language
3. Show derivation chain (*x*) or tree(*x* and *y*) for instance of proof using recursive definition
4. Given a recursive definition of a language, prove a certain characteristic
5. Give a recursive definition, list the invalid substrings of length 2 or 3
6. Given a recursive definition, determine cardinalities based on combinations and permutations

## Problem 3.1:

---
Write another recursive definition for the language $L_1$ of Chapter 2.

$L_1 = \{x\}^+$

**Answer 3.1:**

* Rule 1: $x$ is in $L_1$.
* Rule 2: If $w$ is in $L_1$, then so is $wx$.

## Problem 3.2:

---
Using the second recursive definition of the set EVEN, how many different ways can we prove that 14 is in EVEN?
* Rule 1: 2 is in EVEN.
* Rule 2: If *x* and *y* are both in EVEN, then so is $x+y$.

**Answer 3.2:**

* 2+2=4
  * 2+4=6
    * 2+6=8
      * 2+8=10
        * 2+10=12
          * **2+12=14**
        * **4+10=14**
      * 4+8=12
        * **2+12=14**
      * **6+8=14**
    * 4+6=10
      * 2+10=12
        * **2+12=14**
      * **4+10=14**
    * 6+6=12
      * **2+12=14**
  * 4+4=8
    * 2+8=10
      * 2+10=12
        * **2+12=14**
      * **4+10=14**
    * 4+8=12
      * **2+12=14**
    * **6+8=14**

## Problem 3.3:

---
Using the second recursive definition of EVEN.
* What is the smallest number of steps required to prove that 100 is EVEN?
* Describe a good method for showing that $2n$ is in EVEN.

**Answer 3.3:**
- 2, 2+2=4, 4+4=8, 8+8=16, 16+16=32, 32+32=64, 64+32=96, 96+4=100, eight steps
- Continue doubling until overreach. Back up one. Continue adding highest value in set which will not cause overreach. Repeat until target is reached.

## Problem 3.4:

---
Show that the following is another recursive definition of the set EVEN:
* Rule 1: 2 and 4 are in EVEN
* Rule 2: If $x$ is in EVEN, then so is $x + 4$.

**Answer 3.4:**

We define the set $E$ recursively as follows:

1. **Base cases:** $2 \in E$ and $4 \in E$.
2. **Recursive step:** If $x \in E$, then $x + 4 \in E$.

**Step 1: Show $E \subseteq \text{EVEN}$.**

We proceed by structural induction on the definition of $E$:

- **Base cases:** $2$ and $4$ are even numbers.
- **Inductive step:** Assume $x$ is even. Then $x + 4$ is also even, since the sum of two even numbers is even.

Thus, every element of $E$ is even.

**Step 2: Show $\text{EVEN} \subseteq E$.**

Let $n$ be an even integer with $n \ge 2$.

We can write $n$ as either $n = 2 + 4k \quad \text{or} \quad n = 4 + 4k \quad$ for some integer $k \ge 0$.

- If $n = 2$ or $n = 4$, then $n \in E$ by the base cases.
- If $n > 4$, then $n - 4$ is even and $\ge 2$.  
  By the induction hypothesis, $n - 4 \in E$, and by the recursive step, $n \in E$.

**Step 3: Conclusion**

We have shown: $E \subseteq \text{EVEN} \quad \text{and} \quad \text{EVEN} \subseteq E$

Therefore: $E = \{ n \in \mathbb{N} \mid n \text{ is even and } n \ge 2 \}$.

## Problem 3.5:

---
Show that there are infinitely many different recursive definitions for the set EVEN.

**Answer 3.5:**

$
\textbf{Theorem:} \quad
\text{There are infinitely many distinct recursive definitions of the set } \mathrm{EVEN}.
$

$
\textbf{Definition for a fixed } k \ge 1:
\quad
\begin{cases}
\text{Base cases:} & 2, 4, \dots, 2k \in E_k, \\
\text{Recursive step:} & x \in E_k \ \Rightarrow\ x + 2k \in E_k.
\end{cases}
$

$\textbf{Claim:} \quad E_k = \mathrm{EVEN} \quad \text{for all } k \ge 1.$

$\textit{Proof:}$

$\text{(1) Soundness: All base elements are even. Adding $2k$ preserves evenness, so } E_k \subseteq \mathrm{EVEN}.$

$
\text{(2) Completeness: Let $n \ge 2$ be even. Write $n = 2q$. By the division algorithm,}
$

$$q = kq' + j, \quad \text{with } q' \ge 0, \ j \in \{1,2,\dots,k\}.$$

$\text{Then } n = 2j + 2kq'. \ \text{Since $2j$ is a base element and we can add $2k$ repeatedly, } n \in E_k.$

$$\therefore \ E_k = \mathrm{EVEN} \ \text{for all } k \ge 1.$$

$$
\textbf{Infinitely many definitions:} \quad
\text{Different $k$ give different base sets and step sizes, so the rules differ.}
$$

$\text{Since $k$ can be any positive integer, there are infinitely many such recursive definitions.}$

## Problem 3.6:

---
Using any recursive definition of the set EVEN, show that all the numbers in it end in the digits 0, 2, 4, 6, or 8.

**Answer 3.6:**

We define the set $\mathrm{EVEN}$ recursively as:

1. **Base case:** $2 \in \mathrm{EVEN}$.
2. **Recursive step:** If $x \in \mathrm{EVEN}$, then $x + 2 \in \mathrm{EVEN}$.
3. **Closure:** No other elements are in $\mathrm{EVEN}$.

**Claim:** Every $n \in \mathrm{EVEN}$ ends with digit $0, 2, 4, 6,$ or $8$.

**Proof (structural induction):**

**Base case:**  
$2$ ends with digit $2 \in \{0,2,4,6,8\}$.

**Inductive step:**  
Assume $x$ ends with digit $d \in \{0,2,4,6,8\}$.  
Consider $x + 2$:

$$
\begin{aligned}
d &= 0 \implies d+2 = 2, \\
d &= 2 \implies d+2 = 4, \\
d &= 4 \implies d+2 = 6, \\
d &= 6 \implies d+2 = 8, \\
d &= 8 \implies d+2 = 10 \ (\text{last digit } 0).
\end{aligned}
$$

In each case, the last digit remains in $\{0,2,4,6,8\}$.

**Conclusion:**  
By induction, all elements of $\mathrm{EVEN}$ have last digit in $\{0,2,4,6,8\}$.

## Problem 3.7:

---
The set POLYNOMIAL defined in this chapter contains only the polynomials in the one variable $x$. Write a recursive definition for the set of all polynomials in the two variables $x$ and $y$.

* Rule 1: Any number is in POLYNOMIAL.
* Rule 2: The variable *x* is in POLYNOMIAL.
* Rule 3: If *p* and *q* are in POLYNOMIAL, then so are $p+q$, $p-q$, $(p)$, and $pq$.

**Answer 3.7:**

- Rule 1: Any number is in POLYNOMIAL.
- Rule 2: The variable *x* or *y* is in POLYNOMIAL.
- Rule 3: If *p* and *q* are in POLYNOMIAL, then so are $p+q$, $p-q$, $(p)$, and $pq$.

## Problem 3.8:

---
Define the set of valid algebraic expressions ALEX as follows:
* Rule 1: All polynomials are in ALEX.
* Rule 2: If $f(x)$ and $g(x)$ are in ALEX, then so are:
* (i) $(f(x))$
* (ii) $-(f(x))$
* (iii) $f(x) + g(x)$
* (iv) $f(x) - g(x)$
* (v) $f(x)g(x)$
* (vi) $f(x)/g(x)$
* (vii) $f(x)^{g(x)}$
* (viii) $f(g(x))$

- (a) Show that $(x + 2)^{3x}$ is in ALEX.
- (b) Show that elementary calculus contains enough rules to prove the theorem that all algebraic expressions can be differentiated.
- (c) Is Rule 2(viii) really necessary?

**Answer 3.8:**

- (a) Steps for proof:
  - *Rule 1.2:* $x$
  - *Rule 1.1:* $2$
  - *Rule 1.1:* $3$
  - *Rule 1.3:* $x+2$
  - *Rule 1.3:* $(x+2)$
  - *Rule 1.3:* $3x$
  - *Rule 2:* $(x+2)^{3x}$
- (b) Rules of differentiation exists for all unary and binary operators of POLYNOMIAL.
- (c) No.

## Problem 3.9:

---
* Using the fact that $3x^2 + 7x - 9 = (((((3)x) + 7)x) - 9)$, show how to produce this polynomial from the rules for POLYNOMIAL using multiplication only twice.
* What is the smallest number of steps needed for producing $x^8 + x^4$?
* What is the smallest number of steps needed for producing $7x^7 + 5x^5 + 3x^3 + x$?

**Answer 3.9:**
- The product of $(3)$ and $x$, and the product of $(((3)x) + 7)$ and $x$.
- Steps: $x$, $8$, $4$, $x^8$, $x^4$, $x^8 + x^4$, six steps.
- Steps:
  - 1: $7$
  - 2: $x$
  - 3: $5$
  - 4: $3$
  - 5: $x^7$
  - 6: $x^5$
  - 7: $x^3$
  - 8: $7x^7 + 5x^5$
  - 9: $7x^7 + 5x^5 + 3x^3$
  - 10: $7x^7 + 5x^5 + 3x^3 + x$

## Problem 3.10:

---
Show that if $n$ is less that 31, then $x^n$ can be shown to be in POLYNOMIAL in fewer that eight steps.

**Answer 3.10:**

## Problem 3.11:

---
In this chapter, we mentioned several substrings of length 2 that cannot occur in arithmetic expressions, such as *(/*, *+)*, *//*, and *\*/*. What is the complete list of substrings of length 2 that cannot occur?

**Answer 3.11:**
- *++* *-+* *\*+* */+* *(+*
- *)(* *+\** *-\** */\** *(\**
- *+/* *-/* *\*/* *//* *(/*
- *+)* *-)* *\*)* */)* *()*

## Problem 3.12:

---
Are there any substrings of length 3 that cannot occur that do not contain forbidden substrings of length 2? (This means that *///* is already known to be illegal because it contains the forbidden substring *//*.) What is the longest forbidden substring that does not contain a shorter forbidden substring?

**Answer 3.12:**
Yes. *+--* *---* *\*--* */--* *(--* *\*\*\**

## Problem 3.13:

---
The rules given earlier for the set AE allow for the peculiar expressions

$(((((9)))))$ and $-(-(-(-(9))))$

It is not really harmful to allow these in AE, but is there some modified definition of AE that eliminates this problem?

**Answer 3.13:** Yes, but with some difficulty. Though, it's almost always better to remove the redunancies and equivalencies in a later processing step.

## Problem 3.14:

---
* (i) Write out the full recursive definition for the propositional calculus that contains the symbols $\vee$ and $\wedge$ as well as $¬$ and $→$.
* (ii) What are all the forbidden substrings of length 2 in this language?

**Answer 3.14:**
- (i) PROPCALC is defined as
  - Rule 1: Any letter is in PROPCALC.
  - Rule 2: If *x* and *y* are in PROPCALC, then so are the following:
    - $¬x$
    - $(x)$
    - $x\vee y$
    - $x\wedge y$
    - $x→y$

- (ii) 22 forbidden substrings of length 2
  - $)($, $)¬$
  - $()$, $(\vee$, $(\wedge$, $(→$
  - $¬)$, $¬\vee$, $¬\wedge$, $¬→$
  - $\vee)$, $\vee\vee$, $\vee\wedge$, $\vee →$
  - $\wedge)$, $\wedge\vee$, $\wedge\wedge$, $\wedge →$
  - $→)$, $→\vee$, $→\wedge$, $→→$

## Problem 3.15:

---
* (i) When asked to give a recursive definition for the language PALINDROME over the alphabet $Σ = \{a\, b\}$, a student wrote:
  * Rule 1: *a* and *b* are in PALINDROME
  * Rule 2: If *x* is in PALINDROME, then so are *axa* and *bxb*.

Unfortunately, all the words in the language defined above have an odd length and so it is not all of PALINDROME.
* (i)  Fix this problem.
* (ii) Give a recursive definition for the language EVEN_PALINDROME of all palindromes of even length.

**Answer 3.15:**

- (i) PALINDROME is defined as
  - Rule 1: *a*, *b*, and $Λ$ are in PALINDROME
  - Rule 2: If *x* is in PALINDROME, then so are $xx$, $axa$, and $bxb$.

- (ii) EVEN_PALINDROME is define as
  - Rule 1: *aa*, *bb* are in EVEN_PALINDROME
  - Rule 2: If *x* is in EVEN_PALINDROME, then so are $axa$ and $bxb$.

## Problem 3.16:

---
* (i) Give a recursive definition for the set $ODD = \{1\, 3\, 5\, 7\, ...\}$.
* (ii) Give a recursive definition for the set of strings of digits 0, 1, 2, 3, ..., 9 that cannot start with the digit 0.

**Answer 3.16:**
- $ODD$ is defined as
  - Rule 1: 1 is in $ODD$
  - Rule 2: If *x* is in $ODD$, then so is $x + 2$.

- $DIGITS$ is defined as
  - Rule 1: $\{1\,2\,3\,4\,5\,6\,7\,8\,9\} \subset DIGITS$.
  - Rule 2: If $x$ and $y$ are in $DIGITS$, then so is $x0$ and $xy$.

## Problem 3.17:

---
In this chapter we attempted to define the positive numbers by the following rules:
* Rule 1: 1 is in $L$
* Rule 2: If $x$ and $y$ are in $L$, Then so are $x+y$, $x*y$, and $x/y$.

The language $L$ defined in this way is a famous mathematical set.
* (i) What is it?
* (ii) Prove it.

**Answer 3.17:**
- (i) The set of positive rational numbers.
- (ii) The additional operator gives the positive natural numbers. The division operators gives the fractional rational numbers.

## Problem 3.18:

---
Give two recursive definitions for the set POWERS-OF-TWO $= \{1\, 2\, 4\, 8\, 16\, ...\}$.

Use one of them to prove that the product of two POWERS-OF-TWO is also a POWERS-OF-TWO.

**Answer 3.18:**

- $POWERS\_OF\_TWO$ is defined as
  - Rule 1: $1 \in POWERS\_OF\_TWO$
  - Rule 2: $x \in POWERS\_OF\_TWO \implies 2x \in POWERS\_OF\_TWO$

Claim: $x, y \in POWERS\_OF\_TWO \implies x*y \in POWERS\_OF\_TWO$

Proof:

Given $x, y \in POWERS\_OF\_TWO$.

Recall $x=2^m$ and $y=2^n$, for some m, n.

So $x*y = 2^m2^n = 2^{m+n}$, a power of 2.

$\therefore x*y \in POWERS\_OF\_TWO$.

## Problem 3.19:

---
Give recursive definitions for the the following languages over the alphabet $\{a\, b\}$:
* (i) The language EVENSTRING of all words of even length.
* (ii) The language ODDSTRING of all words of odd length.
* (iii) The language AA of all words containing the substring *aa*.
* (iv)  The language NOTAA of all words not containing the substring *aa*.

**Answer 3.19:**
- (i) $EVENSTRING$ is defined as
  - Rule 1: $Λ \in EVENSTRING.$
  - Rule 2: $w \in EVENSTRING \implies waa$, $wab$, $wba$, $wbb \in EVENSTRING$.
- (ii) $ODDSTRING$ is defined as
  - Rule 1: $a, b \in ODDSTRING$.
  - Rule 2: $w \in ODDSTRING \implies waa$, $wab$, $wba$, $wbb \in ODDSTRING$.
- (iii) $AA$ is defined as
  - Rule 1: $aa \in AA$.
  - Rule 2: $w \in AA \implies wa, aw, wb, bw \in AA$.
- (iv) $NOTAA$ is defined as
  - Rule 1: $Λ, a, b \in NOTAA$.
  - Rule 2: $w \in NOTAA \implies wb, wba \in NOTAA$.

## Problem 3.20:

---
* (i) Consider the following recursive definition of 3-PERMUTATION:
  * Rule 1: 123 is a 3-PERMUTATION.
  * Rule 2: If *xyz* is a 3-PERMUTATION, then so are *zyx* and *yzx*.

  Show that there are six different 3-PERMUTATIONs.

* (ii)
  * Rule 1: 1234 is a 4-PERMUTATION.
  * Rule 2: If *xyzw* is a 4-PERMUTATION, then so are *wzyx* and *yzwx*.

  How many 4-PERMUTATIONs are there (by this definition)?

**Answer 3.20:**
- (i) 123, 321, 231, 213, 312, 132
- (ii) 1234, 4321, 2341, 3214, 1432, 3412, 4123, 2143

# **Chapter 4: Regular Expressions**

## **Functions:**

### Utility

In [ ]:
import textwrap
def print_list(lst, width=80):
    print(textwrap.fill(" ".join(lst), width))

### REX to RE Converter

In [ ]:
def cvt_REX2re(t):
    if t is None: return ""
    assert isinstance(t, list)
    match t[0]:
        case 'σ':   return t[1]
        case '.':   return "(a+b)"
        case '()':  return f"({cvt_REX2re(t[1])})"
        case 'Σ':   return "".join([cvt_REX2re(c) for c in t[1:]])
        case 'Π':   return "+".join([cvt_REX2re(c) for c in t[1:]])
        case 'ς':   # quantifier
                    match t[2]:
                        case ['*']: return cvt_REX2re(t[1]) + "^{*}"
                        case ['+']: return cvt_REX2re(t[1]) + "^{+}"
                        case ['?']: return f"Λ+{cvt_REX2re(t[1])}"
                        case _: raise Exception(f"Unexpected REX quantifier {t[2]}")
        case _:     raise Exception(f"Unknown REX type {t[0]}")

### REX Parser (Python regular expressions)

In [ ]:
REX_tree = None
#------------------------------------------------------------------------------
rex_quantifier  =   ( σ('*') + Shift('*')
                    | σ('+') + Shift('+')
                    | σ('?') + Shift('?')
                    )
rex_item        =   ( σ('.') + Shift('.')
                    | σ('\\') + ANY('.\\(|*+?)') % 'tx' + Shift('σ', "tx")
                    | ANY(UCASE + LCASE + DIGITS) % 'tx' + Shift('σ', "tx")
                    | σ('(') + ζ(lambda: rex_expression) + σ(')') + Reduce('()', 1)
                    )
rex_factor      =   rex_item + (rex_quantifier + Reduce('ς', 2) | ε())
rex_term        =   nPush() + ARBNO(rex_factor + nInc()) + Reduce('Σ') + nPop()
rex_expression  =   ( nPush()
                    + rex_term + nInc()
                    + ARBNO(σ('|') + rex_term + nInc())
                    + Reduce('Π')
                    + nPop()
                    )
rex_parse       =   POS(0) + rex_expression + Pop('REX_tree') + RPOS(0)
#------------------------------------------------------------------------------
def parse_rex2re(rex):
    if rex in rex_parse:
        return f"${cvt_REX2re(REX_tree)}$"
    else: raise Exception("REX parse error")

### RE to REX Converter

In [ ]:
#------------------------------------------------------------------------------
def cvt_RE2rex(t):
    if t is None: return ""
    assert isinstance(t, list)
    match t[0]:
        case 'Λ':   return ""
        case 'σ':   return t[1]
        case '()':  return f"({cvt_RE2rex(t[1])})"
        case 'Σ':   return "".join([cvt_RE2rex(c) for c in t[1:]])
        case 'Π':   return "|".join([cvt_RE2rex(c) for c in t[1:]])
        case 'ς':   # quantifier
                    match t[2]:
                        case ['*']: return f"{cvt_RE2rex(t[1])}*"
                        case ['+']: return f"{cvt_RE2rex(t[1])}+"
                        case _: raise Exception(f"Unexpected RE quantifier {t[2]}")
        case _:     raise Exception(f"Unknown RE type {t[0]}")
#------------------------------------------------------------------------------

### RE Parser (regular expressions)

In [ ]:
RE_tree = None
#------------------------------------------------------------------------------
re_quantifier   =   ( σ('^{*}') + Shift('*')
                    | σ('^{+}') + Shift('+')
                    )
re_item         =   ( σ('Λ') + Shift('Λ')
                    | ANY(UCASE + LCASE + DIGITS) % 'tx' + Shift('σ', "tx")
                    | σ('(') + ζ(lambda: re_expression) + σ(')') + Reduce('()', 1)
                    )
re_factor       =   re_item + (re_quantifier + Reduce('ς', 2) | ε())
re_term         =   nPush() + ARBNO(re_factor + nInc()) + Reduce('Σ') + nPop()
re_expression   =   ( nPush()
                    + re_term + nInc()
                    + ARBNO(σ('+') + re_term + nInc())
                    + Reduce('Π')
                    + nPop()
                    )
re_parse        =   ( POS(0)
                    + σ('$')
                    + re_expression + Pop('RE_tree')
                    + σ('$')
                    + RPOS(0)
                    )
#------------------------------------------------------------------------------
def parse_re2rex(xre):
    if xre in re_parse:
        return f"^({cvt_RE2rex(RE_tree)})$"
    else: raise Exception("RE parse error")

### permutations()

In [ ]:
def permutations(alphabet, max_length=7):
    for length in range(0, max_length+1):
        for combo in product(alphabet, repeat=length):
            yield ''.join(combo)

### show_results()

In [ ]:
def show_results(results, max_width=60):
    base = len(Alphabet)  # 2 for ab, 3 for abc
    REDDISH = "\033[38;2;206;145;120m"   # #ce9178
    GREENISH = "\033[38;2;130;183;108m"  # #82b76c
    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    RESET = "\033[0m"
    sep_len = 1
#   ---------------------------------------------------------------------------
    def colorize(m, a, s):
        return ( (GREENISH if (m is None and a) or m is True else REDDISH)
               + (ITALIC + BOLD if m is not None and a and m != a else "")
               + (s if s != "" else BOLD + "Λ")
               + RESET
               )
    def largest_power_leq(bound, base):
        if base == 1: return 1
        if bound < 1: return 1
        exp = int(math.log(bound, base))
        return base ** exp if base ** exp <= bound else base ** (exp - 1)
#   ---------------------------------------------------------------------------
    for length, bucket in enumerate(results):
        if not bucket: continue
        denom = length + sep_len
        numer = max_width - sep_len
        max_items = max(1, numer // denom)
        group_size = largest_power_leq(min(max_items, len(bucket)), base)
        for i in range(0, len(bucket), group_size):
            group = bucket[i:i+group_size]
            items = []
            for item in group:
                m, a, s = item
                items.append(colorize(m, a, s))
            print(" ".join(items))

### test_re()

In [ ]:
Alphabet = "ab"
def test_re(rex, max_length, verbose):
    results = []
    if regex := re.compile(rex):
        for s in permutations(Alphabet, max_length):
            length = len(s)
            if length + 1 > len(results):
                results.append(list())
            m = re.match(regex, s) is not None
            results[length].append((m, None, s if s != "" else "Λ"))
    else: raise Exception("re.compile error")
    return results

### display_re_problem()

In [ ]:
def display_re_problem(xre, max_length, verbose=False):
    display(Latex(xre))
    rex = parse_re2rex(xre) if xre != "" else ""
    if rex != "": print(rex)
    results = test_re(rex, max_length, verbose)
    show_results(results)

## Problem 4.1:

---
Let $r_1$, $r_2$, and $r_3$ be three regular expressions.
Show that the language associated with $(r_1+r_2)r_3$ is the same as the language associated with $r_1r_3+r_2r_3$.
Show that $r_1(r_2+r_3)$ is equivalent to $r_1r_2+r_1r_3$.
This will be the same as proving a "distributive law" for regular expressions.

**Answer 4.1:**

Let $L(r)$ be the language denoted by the regular expression $r$.

- Union: $L(r_1 + r_2) = L(r_1) \cup L(r_2)$
- Concatenation: $L(r_1 r_2) = \{\, xy \mid x \in L(r_1),\ y \in L(r_2) \,\}$

Claim: $$L\big((r_1 + r_2) r_3\big) = L(r_1 r_3) \cup L(r_2 r_3).$$

Proof:
- ($\subseteq$) Let $w \in L((r_1 + r_2) r_3)$.
  - Then $w = xy$ with $x \in L(r_1) \cup L(r_2)$ and $y \in L(r_3)$.
  - If $x \in L(r_1)$, then $w \in L(r_1 r_3)$.
  - If $x \in L(r_2)$, then $w \in L(r_2 r_3)$.
  - Hence $w \in L(r_1 r_3) \cup L(r_2 r_3)$.

- ($\supseteq$) Let $w \in L(r_1 r_3) \cup L(r_2 r_3)$.
  - If $w \in L(r_1 r_3)$, then $w = xy$ with $x \in L(r_1)$ and $y \in L(r_3)$.
  - Since $x \in L(r_1) \subseteq L(r_1) \cup L(r_2)$, we have $w \in L((r_1 + r_2) r_3)$.
  - The case $w \in L(r_2 r_3)$ is analogous.

Therefore, $$L((r_1 + r_2) r_3) = L(r_1 r_3) \cup L(r_2 r_3).$$

Conclusion:
$$
(r_1 + r_2) r_3 \equiv r_1 r_3 + r_2 r_3
\quad\text{and}\quad
r_1 (r_2 + r_3) \equiv r_1 r_2 + r_1 r_3.
$$

## For Problems 4.2 through 4.11

construct a regular expression defining each of the following languages over the alphabet $Σ = \{a\,b\}$.

A recursive definition for REGEX is:
* Rule 1: $Λ, a, b \in REGEX$.
* Rule 2: If $r_1, r_2 \in REGEX$, then $(r_1), r_1r_2, r_1+r_2, r_1^{*} \in REGEX$.

## Problem 4.2:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all words in which *a* appears tripled, if at
all. This means that every clump of *a*'s contains 3 or 6 or 9 or 12 ... *a*'s.

In [ ]:
# Answer 4.2:
display_re_problem("$(aaa+b)^{*}$", 6, True)
display_re_problem("$((aaa)^{*}b^{*})^{*}$", 6, True)
display_re_problem("$(b^{*}(aaa)^{*})^{*}$", 6, True)

## Problem 4.3:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all words that contain at least one of the strings $s_1$, $s_2$, $s_3$, or $s_4$.

**Answer 4.3:** $(a+b)^{*}(s_1+s_2+s_3+s_4)(a+b)^{*}$

## Problem 4.4:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all words that contain exactly two *b*'s or exactly three *b*'s, not more.

In [ ]:
# Answer 4.4:
display_re_problem("$a^{*}ba^{*}ba^{*}(Λ+ba^{*})$", 5, True)

## Problem 4.5:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for
* (i) all strings that end in a double letter,
* (ii) all strings that do not end in a double letter.

In [ ]:
# Answer 4.5:
display_re_problem("$(a+b)^{*}(aa+bb)$", 5, True)
display_re_problem("$Λ+a+b+(a+b)^{*}(ab+ba)$", 5, True)

## Problem 4.6:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all strings that have exactly one double letter in them.

In [ ]:
# Answer 4.6:
display_re_problem("$(Λ+b)(ab)^{*}aa(ba)^{*}(Λ+b)+(Λ+a)(ba)^{*}bb(ab)^{*}(Λ+a)$", 5, True)

## Problem 4.7:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all strings in which the letter *b* is *never* tripled. This means that no word contains the string *bbb*.

In [ ]:
# Answer 4.7:
display_re_problem("$(Λ+b+bb)(a^{*}a(b+bb))^{*}a^{*}$", 6, True)

## Problem 4.8:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all words in which *a* is tripled or *b* is tripled, but not both. This means each word contains the substring *aaa* or the substring *bbb* but not both.

In [ ]:
# Answer 4.8:
NOTBBB="(Λ+b+bb)(a^{+}(b+bb))^{*}a^{*}"
HASAAA=f"{NOTBBB}aaa{NOTBBB}"
NOTAAA="(Λ+a+aa)(b^{+}(a+aa))^{*}b^{*}"
HASBBB=f"{NOTAAA}bbb{NOTAAA}"
display_re_problem(f"${HASAAA}+{HASBBB}$", 6, True)

## Problem 4.9:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for
* (i) all words that do not have the substring *ab*,
* (ii) all words that do not have both the substrings *bba* and *abb*.

In [ ]:
# Answer 4.9(i):
display_re_problem("$b^{*}a^{*}$", 6, True)

In [ ]:
# Answer 4.9(ii):
#HAS_ABB="(a+b)^{*}abb(a+b)^{*}"
#NOT_ABB="(Λ+a+b)(Λ+a+b)+b^{*}(a^{+}b)^{*}a^{*}"
#HAS_BBA="(a+b)^{*}bba(a+b)^{*}"
#NOT_BBA="(Λ+a+b)(Λ+a+b)+a^{*}(ba^{+})^{*}b^{*}"
display_re_problem("$(a+ba)^{*}b^{*}+b^{*}a(a+ba)^{*}(Λ+b)$", 6, True)

## Problem 4.10:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all strings in which the *total* number of *a*'s is divisible by 3 no matter how they are distributed, such as *aabaabbaba*.

In [ ]:
# Answer 4.10:
display_re_problem("$b^{*}(ab^{*}ab^{*}a)^{*}b^{*}$", 6, True)

## Problem 4.11:

---
Given $Σ = \{a\,b\}$. Construct a regular expression for all strings

* (i) in which any *b*'s that occur are found in clumps of an odd number at a time, such as *abaabbbab*.
* (ii) that have an even number of *a*'s and an odd number of *b*'s.
* (iii) that have an odd number of *a*'s and an odd number of *b*'s.

In [ ]:
# Answer 4.11:
EVEN_EVEN = "(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}"
display_re_problem("$a^{*}+a^{*}(b(bb)^{*}a^{+})^{*}b(bb)^{*}a^{*}$", 5, True)
display_re_problem(f"${EVEN_EVEN}" + "(b+ab(bb)^{*}a)" + f"{EVEN_EVEN}$", 5, True)
display_re_problem(f"${EVEN_EVEN}" + "(ab+ba)" + f"{EVEN_EVEN}$", 5, True)

## Problem 4.12:

---

* (i) Let us reconsider the regular expression $$(a+b)^{*}a(a+b)^{*}b(a+b)^{*}$$ Show that this is equivalent to $$(a+b)^{*}ab(a+b)^{*}$$ in the sense that they define the same language.
* (ii) Show that $(a+b)^{*}ab(a+b)^{*} + b^{*}a^{*} = (a+b)^{*}$.
* (iii) Show that $(a+b)^{*}ab(a+b)^{*}ab(a+b)^{*} = (a+b)^{*}$.
* (iv) Is (iii) the last variation of this theme or are there more beasts left in this cave?


**Answer 4.12(i):**

Let $L_1 = L\big((a+b)^{*}a(a+b)^{*}b(a+b)^{*}\big).$

Let $L_2 = L\big((a+b)^{*}ab(a+b)^{*}\big).$

Take any $w \in L_1$. Then $w$ can be written as  
$$
w = x \; a \; m \; b \; y
\quad\text{with}\quad
x,y \in \{a,b\}^{*}, \; m \in \{a,b\}^{*}.
$$

We analyze the possible forms of $m$:

- **Case 1: $m = \epsilon$**  
  Then $w = x\,ab\,y$, so $w$ contains the substring $ab$ and hence $w \in L_2$.

- **Case 2: $m \neq \epsilon$ and $m$ contains a $b$**  
  Let the first $b$ in $m$ occur at position $j$.  
  - If $j = 1$, then the boundary between $a$ and the first symbol of $m$ is $ab$, so $w$ contains $ab$.  
  - If $j > 1$, then the symbol at position $j-1$ in $m$ cannot be $b$ (by minimality of $j$), so it is $a$, giving an internal $ab$ in $m$.  
  In both subcases, $w \in L_2$.

- **Case 3: $m$ has no $b$**  
  Then $m = a^{k}$ for some $k \ge 1$. The last $a$ of $m$ is immediately followed by the trailing $b$, so $ab$ occurs in $w$. Thus $w \in L_2$.

In all cases, $w$ contains a contiguous $ab$, so $w \in L_2$.

If $w \in L_2$, then  
$$w = x\,ab\,y, \quad x,y \in \{a,b\}^{*}.$$

This matches $(a+b)^{*}a(a+b)^{*}b(a+b)^{*}$ by choosing $m = \epsilon$. Hence $w \in L_1$.

Since $L_1 \subseteq L_2$ and $L_2 \subseteq L_1$, we have  
$$L\big((a+b)^{*}a(a+b)^{*}b(a+b)^{*}\big)=L\big((a+b)^{*}ab(a+b)^{*}\big).$$

**Answer 4.12(ii):**

Since $(a+b)^{*}ab(a+b)^{*}ab(a+b)^{*}$ represents all strings which have a *b* following an *a*. This exludes from the set of all possible strings, $a^{*}$, $b^{*}$, and $b^{*}a^{*}$. Hence the union of the two is the set of all possible string $(a+b)^{*}$.

**Answer 4.12(iii):**

Strictly speaking there are an infinite amount of these type of substitutions.

## Problem 4.13:

---
We have defined the product of two sets of strings in general. If we apply this to the case where both factors are the same set, $S=T$, we obtain squares, $S^2$. Similarily, we can define $S^3, S^4,...$. Show that it makes some sense to write:
* (i) $S^{*} = Λ + S + S^1 + S^2 + S^3 + S^4 + ...$
* (ii) $S^{+} = S + S^1 + S^2 + S^3 + S^4 + ...$

**Answer 4.13:**

- (i) Obvious.
- (ii) Obvious.

## Problem 4.14:

---
If the only difference between $L$ and $L^{*}$ is the word $Λ$, is the only difference between $L^2$ and $L^{*}$ the word $Λ$?

**Answer 4.14:**

Let $L=(a+b)^{+}$. $L$ and $L^{*}$ differ only by $Λ$.

Now $L^2=(a+b)(a+b)^{+}$. So, no. $L$ and $L^{*}$ differ by $Λ$, $a$, and $b$.

## Problem 4.15:

---
Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $(ab)^{*}a\quad$ and $\quad a(ba)^{*}$
* (ii) $(a^{*}+b)^{*}\quad$ and $\quad (a+b)^{*}$
* (iii) $(a^{*}+b^{*})^{*}\quad$ and $\quad (a+b)^{*}$

**Answer 4.15(i):**

Let the alphabet be $\Sigma = \{a,b\}$ with concatenation, associativity and the identity $\epsilon$.

Define  
- $w_0 = \epsilon$, $w_{k+1} = w_k\,ab$ so $w_k = (ab)^k$,  
- $v_0 = \epsilon$, $v_{k+1} = v_k\,ba$ so $v_k = (ba)^k$.

**Claim:** for all $k \ge 0$, $w_k\,a = a\,v_k$.

**Base case ($k=0$):** $\epsilon a = a = a\epsilon.$

**Inductive step:**

Assume $w_k a = a v_k$.

Then  
$$
\begin{aligned}
w_{k+1} a
&= (w_k\,ab)\,a \\
&= w_k\,(ab\,a) && \text{(associativity)} \\
&= (w_k a)\,ba \\
&= (a v_k)\,ba && \text{(inductive hypothesis)} \\
&= a\,(v_k\,ba) \\
&= a\,v_{k+1}.
\end{aligned}
$$

Thus $w_k a = a v_k$ for all $k$, i.e., $(ab)^k a = a(ba)^k$.

**Answer 4.15(ii)**

We work over the alphabet $\Sigma = \{a, b\}$. The claim is  
$$
(a^{*} + b)^{*} \;=\; \Sigma^{*} \;=\; (a + b)^{*}.
$$

**Claim:** Every $w \in \Sigma^{*}$ belongs to $(a^{*} + b)^{*}$.

**Base case: ($|w| = 0$):**  
$$
w = \epsilon \in (a^{*} + b)^{*}.
$$

**Inductive step:** Assume all words of length $\le n$ are in $(a^{*} + b)^{*}$. Take $w$ with $|w| = n+1$.

- **Case 1:** $w$ starts with $b$. Write $w = b\,u$. By the inductive hypothesis, $u \in (a^{*} + b)^{*}$, hence $w \in (a^{*} + b)^{*}$ since $b \in a^{*} + b$.

- **Case 2:** $w$ starts with $a$. Let $w = a^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $b$. Then $a^{k} \in a^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b)^{*}$. Hence $w \in (a^{*} + b)^{*}$.

Thus  
$$
\Sigma^{*} \subseteq (a^{*} + b)^{*},
$$
and the equality with $(a + b)^{*}$ follows as above.

**Answer 4.15(iii)**

We work over the alphabet $\Sigma = \{a, b\}$. The claim is  
$$
(a^{*} + b^{*})^{*} \;=\; \Sigma^{*} \;=\; (a + b)^{*}.
$$

**Claim:** Every $w \in \Sigma^{*}$ belongs to $(a^{*} + b^{*})^{*}$.

**Base case ($|w| = 0$):**  
$$
w = \epsilon \in (a^{*} + b^{*})^{*}.
$$

**Inductive step:** Assume all words of length $\le n$ are in $(a^{*} + b^{*})^{*}$. Take $w$ with $|w| = n+1$.

- **Case 1:** $w$ starts with $b$. Let $w = b^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $a$. Then $b^{k} \in b^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b^{*})^{*}$. Hence $w \in (a^{*} + b^{*})^{*}$.

- **Case 2:** $w$ starts with $a$. Let $w = a^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $b$. Then $a^{k} \in a^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b^{*})^{*}$. Hence $w \in (a^{*} + b^{*})^{*}$.

Thus  
$$
\Sigma^{*} \subseteq (a^{*} + b^{*})^{*},
$$
and the equality with $(a + b)^{*}$ follows as above.

## Problem 4.16:

---
Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $Λ^{*}\quad$ and $\quad Λ$
* (ii) $(a^{*}b)^{*}a^{*}\quad$ and $\quad a^{*}(ba^{*})^{*}$
* (iii) $(a^{*}bbb)^{*}a^{*}\quad$ and $\quad a^{*}(bbba^{*})^{*}$

**Answer 4.16:**

- (i) Obvious.
- (ii) Follows from $(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$.
- (iii) Follows from $(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$.

## Problem 4.17:

---
Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $((a+bb)^{*}aa)^{*}\quad$ and $\quad Λ+(a+bb)^{*}aa$
* (ii) $(aa)^{*}(Λ+a)\quad$ and $\quad a^{*}$
* (iii) $a(aa)^{*}(Λ+a)b+b\quad$ and $\quad a^{*}b$
* (iv) $a(ba+a)^{*}b\quad$ and $\quad aa^{*}b(aa^{*}b)^{*}$
* (v) $Λ+a(a+b)^{*}+(a+b)^{*}aa(a+b)^{*}\quad$ and $\quad ((b^{*}a)^{*}ab^{*})^{*}$

**Answer 4.17:**

- (i) Follows from definition of Kleene closure.
- (ii) A combination of mutiple of two *a*'s followed by an optional *a*, is simply zero or more *a*'s.
- (iii) Follows from (ii) above.
- (iv) Regarding the first, strings begin with *a* and end with *b* and between *b*'s are always followed by an *a*. Or in other words, each *b* is surrounded by at least one *a* on either side.
- (v) $???$

## Problem 4.18:

---
Describe (in English phrases) the languages associated with the following regular expressions:

### $\textbf{Problem 4.18(i)}\;\; (a+b)^{*}a(Λ+bbbb)$
Answer 4.18(i): all strings ending in either *a* or *abbbb*.

In [ ]:
display_re_problem("$(a+b)^{*}a(Λ+bbbb)$", 6, True)

### $\textbf{Problem 4.18(ii)}\;\; (a(a+bb)^{*})^{*}$
Answer 4.18(ii): all strings of repeating sequences of *a*'s followed by an even number of *b*'s.

In [ ]:
display_re_problem("$(a(a+bb)^{*})^{*}$", 6, True)

### $\textbf{Problem 4.18(iii)}\;\; (a(aa)^{*}b(bb)^{*})^{*}$
Answer 4.18(iii): Λ or even-length strings of repeating sequences of odd-number of *a*'s followed by an odd number of *b*'s.

In [ ]:
display_re_problem("$(a(aa)^{*}b(bb)^{*})^{*}$", 8, True)

### $\textbf{Problem 4.18(iv)}\;\; (b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}$
Answer 4.18(iv): optional *b*'s, then optional sequences of odd-length *a*'s followed by odd-length *b*'s.

In [ ]:
display_re_problem("$(b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}$", 5, True)

### $\textbf{Problem 4.18(v)}\;\; (b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}(a(aa)^{*})^{*}$
Answer 4.18(v): optional *b*'s, then optional sequences of odd-length *a*'s followed by odd-length *b*'s, then optional *a*'s.

In [ ]:
display_re_problem("$(b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}(a(aa)^{*})^{*}$", 5, True)

### $\textbf{Problem 4.18(vi)}\;\; ((a+b)a)^{*}$
Answer 4.18(vi): Λ or even-length strings with *a* repeating in even positions.

In [ ]:
display_re_problem("$((a+b)a)^{*}$", 6, True)

## Problem 4.19: (D. N. Arden)

---
Let $R$, $S$, and $T$ be three languages and assume that $Λ$ is not in $S$.

Prove the following statements:
* From the premise that $R=SR+T$, we can conclude that $R=S^{*}T$
* From the premise that $R=S^{*}T$, we can conclude that $R=SR+T$.

**Answer 4.19:**

$R=SR+T$ defines $R$ as containing all the words in $T$ and also all words composed of an initial part from $S$ concatenated with another words from $R$. This permits us to continue concatenating factors from $S$ as many time as we like, but we are still required to finish the word with a factor from $R$. The only type of factor from $R$ that does not entail this kind of recursion is a factor from the language $T$. So we have any number of factors from S followed by a factor from $T$, or we have just a factor from $T$, equivalently $S^{*}T$.

$R=S^{*}T$ defines $R$ as words with any number (if any) factors from $S$ followed by a single factor from $T$. If the star operator is taken fro zero, we have for $R$ any word in $T$. Suppose the star operator is taken once, then we have exactly one word from $S$ followed by one word from $T$. But the word from $T$ is itself a factor from $R$, so we can see that this word is in the set $SR$. Consider what happens if we take the star more than once; the star operates on $S$, so we have $SR, SSR, SSSR, ...$ all in the set $S^{*}T$. In fact, we established $S^{+}T$ in this way. The difference between $S^{*}T$ and $S^{+}T$ is that only $S^{*}T$ contains $T$. But that means that $S^{*}T$ must equal exactly $SR+T$.

**Answer 4.19:**

**Statement and assumptions**

Let $R, S, T \subseteq \Sigma^{*}$ be languages, and assume $\varepsilon \notin S$. Prove:

1. If $R = S R \cup T$, then $R = S^{*} T$.
2. If $R = S^{*} T$, then $R = S R \cup T$.

**Preliminaries**

- **Kleene star:** $S^{*} = \bigcup_{n \ge 0} S^{n}$ and $S^{*} = \{\varepsilon\} \cup S S^{*}$.
- **Length fact (uses $\varepsilon \notin S$):** Every $x \in S$ satisfies $|x| \ge 1$. Hence if $w = xy$ with $x \in S$, then $|y| < |w|$.

From $R = S R \cup T$ to $R = S^{*} T$

We prove both inclusions.

Inclusion $R \subseteq S^{*} T$:

Fix $w \in R$. Proceed by induction on $|w|$.

- **Base case:** If $w \in T$, then $w \in S^{0} T \subseteq S^{*} T$.
- **Inductive step:** If $w \notin T$, then $w \in S R$ by $R = S R \cup T$.  
  So $w = xy$ for some $x \in S$ and $y \in R$.  
  Since $\varepsilon \notin S$, we have $|x| \ge 1$, hence $|y| < |w|$.  
  By the induction hypothesis, $y \in S^{*} T$. Thus  
  $$w = x y \in S S^{*} T \subseteq S^{*} T.$$

Therefore $R \subseteq S^{*} T$.

Inclusion $S^{*} T \subseteq R$:

We show $S^{n} T \subseteq R$ for all $n \ge 0$ by induction on $n$.

- **Base case: ($n=0$):** $S^{0} T = T \subseteq R$ since $R = S R \cup T$.
- **Inductive step:** Suppose $S^{n} T \subseteq R$. Then  
  $$S^{n+1} T = S (S^{n} T) \subseteq S R \subseteq S R \cup T = R.$$

Hence $S^{*} T = \bigcup_{n \ge 0} S^{n} T \subseteq R$.

Combining both inclusions, we have $R = S^{*} T$.

From $R = S^{*} T$ to $R = S R \cup T$:

Using $S^{*} = \{\varepsilon\} \cup S S^{*}$, we have:
$$
R = S^{*} T
  = (\{\varepsilon\} \cup S S^{*}) T
  = T \cup S (S^{*} T)
  = T \cup S R
  = S R \cup T.
$$

This implication does not require $\varepsilon \notin S$.

**Conclusion**

Under $\varepsilon \notin S$, the equation $R = S R \cup T$ holds if and only if $R = S^{*} T$ (Arden’s lemma).

## Problem 4.20:

---
Explain why we can take any pair of equivalent regular expressions and replace the letter *a* in both with any regular expression **R** and the letter *b* with any regular expression **S** and the resulting regular expressions will have the same language. For example, 17(ii), which says

   $$(a^{*}b)^{*}a^{*}=a^{*}(ba^{*})^{*}$$

becomes the identity

   $$(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$$

which is true for all regular expressions **R** and **S**.

**Answer 4.20(i):**

It is easiest to understand this in terms of the recursive definition of regular expressions. Any letter of the alphabet is itself a regular expression. The rule for forming regular expressions apply to these and to all more complicated regular expressions. Each regular expression describes a set. Equivalent expressions describe equivalent sets. The equivalences are not disturbed by taking unions, products, or Kleen closures. This means that any set must be considered a building block in the construction of other sets.

**Answer 4.20(i):**

**Substitution Preserves Equivalence of Regular Expressions**

**1. Setup**
Let $E_1$ and $E_2$ be regular expressions over the alphabet $\{a,b\}$ with  
$$
L(E_1) = L(E_2).
$$

Let $R$ and $S$ be arbitrary regular expressions (over any alphabet).  
We write  
$$
E[a \leftarrow R,\, b \leftarrow S]
$$  
for the expression obtained by replacing each $a$ by $R$ and each $b$ by $S$.

**2. Substitution on Words and Languages**
Define a substitution $\sigma$ on letters by:
$$
\sigma(a) = L(R), \quad \sigma(b) = L(S).
$$

Extend $\sigma$ to words $w = w_1 w_2 \dots w_n \in \{a,b\}^*$ by:
$$
\hat{\sigma}(w) = \sigma(w_1) \cdot \sigma(w_2) \cdots \sigma(w_n),
$$
and to languages $L \subseteq \{a,b\}^*$ by:
$$
\hat{\sigma}(L) = \bigcup_{w \in L} \hat{\sigma}(w).
$$

**3. Lemma — Semantics Commute with Substitution**
For every regular expression $E$ over $\{a,b\}$:
$$
L\big(E[a \leftarrow R,\, b \leftarrow S]\big) = \hat{\sigma}\big(L(E)\big).
$$

**Proof (by structural induction on $E$):**

1. **Empty set**:  
   $E = \emptyset$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = \emptyset = \hat{\sigma}(\emptyset).
   $$

2. **Empty word**:  
   $E = \varepsilon$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = \{\varepsilon\} = \hat{\sigma}(\{\varepsilon\}).
   $$

3. **Letter $a$**:  
   $E = a$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = L(R) = \sigma(a) = \hat{\sigma}(\{a\}).
   $$

4. **Letter $b$**: symmetric to $a$.

5. **Union**: $E = F + G$  
   $$
   L\big((F+G)[\cdot]\big) = L(F[\cdot]) \cup L(G[\cdot])  
   = \hat{\sigma}(L(F)) \cup \hat{\sigma}(L(G))  
   = \hat{\sigma}(L(F) \cup L(G)).
   $$

6. **Concatenation**: $E = FG$  
   $$
   L\big((FG)[\cdot]\big) = L(F[\cdot]) \cdot L(G[\cdot])  
   = \hat{\sigma}(L(F)) \cdot \hat{\sigma}(L(G))  
   = \hat{\sigma}(L(F) \cdot L(G)).
   $$

7. **Kleene star**: $E = F^*$  
   $$
   L\big((F^*)[\cdot]\big) = \big(L(F[\cdot])\big)^*  
   = \big(\hat{\sigma}(L(F))\big)^*  
   = \hat{\sigma}\left(\bigcup_{n \ge 0} L(F)^n\right)  
   = \hat{\sigma}(L(F^*)).
   $$

Thus, the lemma holds.

**4. Theorem — Substitution Preserves Equivalence**
If $L(E_1) = L(E_2)$, then for all $R, S$:
$$
L\big(E_1[a \leftarrow R,\, b \leftarrow S]\big)  
= \hat{\sigma}(L(E_1))  
= \hat{\sigma}(L(E_2))  
= L\big(E_2[a \leftarrow R,\, b \leftarrow S]\big).
$$

Therefore, $E_1[a \leftarrow R,\, b \leftarrow S]$ and $E_2[a \leftarrow R,\, b \leftarrow S]$ are equivalent for all choices of $R, S$.

**Problem 4.20(ii)**

In particular, $$R=a+bb, S=ba^{*}$$

results in the complicated identity

$$((a+bb)^{*}(ba^{*}))^{*}(a+bb)^{*}=(a+bb)^{*}((ba^{*})(a+bb)^{*})^{*}$$

What identity would result from using $$R=(ba^{*})^{*}\quad S=(Λ+b)$$

**Answer 4.20(ii)**

$$(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$$
results in
$$((ba^{*})^{*}(Λ+b))^{*}(ba^{*})^{*}=(ba^{*})^{*}((Λ+b)(ba^{*})^{*})^{*}$$

# **Chapter 5: Finite Automata**

## Functions

### display_fa()

In [ ]:
def display_fa(FA):
    dot = Digraph()
    dot.attr(rankdir='LR')
    dot.attr(bgcolor='black')
    dot.attr('node', style='filled', fillcolor='black', fontcolor='white', color='white')
    dot.attr('edge', color='white', fontcolor='white')
    Σ = FA[0][1:]
    Δ = FA[1:]
    for i, δ in enumerate(Δ):
        q = δ[0]
        is_start = False
        is_finish = False
        if m := re.match(r"^\-(.*)$", q): is_start = True;  q = m.groups(1)[0]
        if m := re.match(r"^\+(.*)$", q): is_finish = True; q = m.groups(1)[0]
        if is_finish: dot.node(str(i+1), shape='doublecircle')
        else:         dot.node(str(i+1), shape='circle')
        if is_start:
            dot.node('', width='0', height='0', margin='0', shape='point', style='invis')
            dot.edge('', str(i+1))
    for i, δ in enumerate(Δ):
        for j in range(1, len(δ)):
            dot.edge(str(i+1), str(δ[j]), label=Σ[j-1])
    dot.render('dfa', format='png', cleanup=False)  # Saves as dfa.png
    display(dot)

### compile_fa()

In [ ]:
def compile_fa(FA):
    header = FA[0][1:]
    states = FA[1:]
    start_state = None
    accepting_states = set()
    transitions = {}
    for row in states:
        state_label = row[0]
        state_num = int(state_label.lstrip('+-'))
        if state_label.startswith('-'):
            start_state = state_num
        if state_label.startswith('+') or state_label.startswith('-+'):
            accepting_states.add(state_num)
        transitions[state_num] = {}
        for symbol, next_state in zip(header, row[1:]):
            transitions[state_num][symbol] = next_state
    if start_state is None:
        raise ValueError("No start state defined in FA table.")
    return start_state, header, transitions, accepting_states

### run_fa()

In [ ]:
def run_fa(string, start_state, header, transitions, final_states):
    current_state = start_state
    for ch in string:
        if ch not in header:
            raise ValueError(f"Invalid input symbol: {ch}")
        current_state = transitions[current_state][ch]
    accepted = current_state in final_states
    return accepted, current_state

### test_fa()

In [ ]:
def test_fa(FA, rex, max_length):
    results = []
    regex = re.compile(rex) if rex else None
    alphabet = FA[0][1:]
    fa = compile_fa(FA)
    for length in range(0, max_length+1):
        for combo in product(alphabet, repeat=length):
            s = ''.join(combo)
            length = len(s)
            if length + 1 > len(results):
                results.append(list())
            m = re.match(regex, s) is not None if regex else None
            a, _ = run_fa(s, *fa)
            results[length].append((m, a, s))
    return results

### display_fa_problem()

In [ ]:
def display_fa_problem(problem, max_length=7, verbose=False):
    fa = problem[2]
    display_fa(fa)
    xre = problem[0]
    if xre:
        display(Latex(xre))
        rex = parse_re2rex(xre)
        print(rex)
    else: rex = None
    results = test_fa(fa, rex, max_length)
    show_results(results)

## Problem 5.1:

---
Write out the transition tables for the FAs on pp. 56, 58 (both), 63, 64, and 69 that were defined by
pictures.

**Answer 5.1:** *See below*

In [ ]:
#-------------------------------------------------------------------------------
FAs['pg_56'] = (\
  "$(a+b)^{*}b(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3]
  , [  '2',  1 ,  3]
  , [ '+3',  3 ,  3]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['pg_56'], 5, True)

In [ ]:
#-------------------------------------------------------------------------------
FAs['pg_58_1'] = (\
  "$(a+b)(a+b)^{*}$"
, SPAN('ab')
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  2 ,  2 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['pg_58_1'])

In [ ]:
#-------------------------------------------------------------------------------
FAs['pg_58_2'] = (\
  "$(a+b)^{*}$"
, SPAN('ab') | ε()
, [ [  ' ', 'a', 'b']
  , ['-+1',  1 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['pg_58_2'])

In [ ]:
#-------------------------------------------------------------------------------
FAs['pg_63'] = (\
  "$(a+b)^{*}(aa+bb)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + (σ('aa') | σ('bb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  4 ,  3 ]
  , [  '3',  2 ,  4 ]
  , [ '+4',  4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['pg_63'])

In [ ]:
#-------------------------------------------------------------------------------
FAs['pg_64'] = (\
  "$(a+b)(a+b)b(a+b)*$"
, (σ('aaa') | σ('bbb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  5 ]
  , [  '4',  4 ,  4 ]
  , [ '+5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['pg_64'])

In [ ]:
# pg_69
na = nb = 0
def init(): na = nb = 0; return True
def acnt(): global na; na += 1; return True
def bcnt(): global nb; nb += 1; return True
FAs['EVEN_EVEN'] = (\
  "$(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}$"
, POS(0) + Λ("init()")
+ ARBNO(
    σ('a') + λ("acnt()")
  | σ('b') + λ("bcnt()")
  )
+ RPOS(0) + λ("a % 2 == 0 and b % 2 == 0")
, [ [  ' ', 'a', 'b']
  , ['-+1',  3 ,  2 ]
  , [  '2',  4 ,  1 ]
  , [  '3',  1 ,  4 ]
  , [  '4',  2 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem('EVEN_EVEN')

## Problem 5.2:

---
Build an FA that accepts only the language of all words with b as the second letter. Show both the picture and the transition table for this machine and find a regular expression for the language.

**Answer 5.2:**

RE = $(a + b)b(a+b)^{*}$

In [ ]:
#-------------------------------------------------------------------------------
FAs['prob5_2'] = (\
  "$(a+b)b(a+b)^{*}$"
, ANY("ab") + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  2 ]
  , [  '2', 3 ,  4 ]
  , [  '3', 3 ,  3 ]
  , [ '+4', 4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_2'])

## Problem 5.3:

---
Build an FA that accepts only the words *ab*, *abb*, and *baa* and no other strings longer or shorter.

**Answer 5.3:**

RE = $(ab(Λ + b)+baa)$

In [ ]:
#-------------------------------------------------------------------------------
FAs['prob5_3'] = (\
  "$(ab(Λ+b)+baa)$"
, σ('baa') | σ('ab') | σ('abb')
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  5 ]
  , [  '2', 8 ,  3 ]
  , [ '+3', 8 ,  4 ]
  , [ '+4', 8 ,  8 ]
  , [  '5', 6 ,  8 ]
  , [  '6', 7 ,  8 ]
  , [ '+7', 8 ,  8 ]
  , [  '8', 8 ,  8 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_3'])

## Problem 5.4:

---
* (i) Build an FA with the three states that accepts all strings.
* (ii) Show that given any FA with three states and three *+*'s, it accepts all input strings.
* (iii) If an FA has three states and only one *+*, must it reject some inputs?

**Answer 5.4:**
- *See below*
- follows from the pigeon-hole principle
- No.

In [ ]:
FAs['prob5_4'] = (\
  "$(a+b)^{*}$"
, ARBNO(σ('a') | σ('b'))
, [ [ ' ' , 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  3 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_4'])

## Problem 5.5:

---
* (i) Build an FA that accepts only those words that have more that four letters.
* (ii) Build an FA that accepts only those words that have fewer than four letters.
* (iii) Build an FA that accepts only those words with exactly four letters.

**Answer 5.5:**
- RE: $\quad (a+b)^{5}(a+b)^{*}$
- RE: $\quad Λ+a+b+(Λ+a+b+(Λ+a+b)), \quad (Λ+a+b)^3$
- RE: $\quad (a+b)(a+b)(a+b)(a+b), \quad (a+b)^4$

In [ ]:
FAs['prob5_5_1'] = (\
  "$(a+b)(a+b)(a+b)(a+b)(a+b)(a+b)^{*}$"
, POS(0) + SPAN("ab") @ "tx" + Λ("len(tx) > 4") + RPOS(0)
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [  '5',  6 ,  6 ]
  , [ '+6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_5_1'])

In [ ]:
FAs['prob5_5_2'] = (\
  "$(Λ+a+b)(Λ+a+b)(Λ+a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) < 4")
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [ '+4',  5 ,  5 ]
  , [  '5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_5_2'])

In [ ]:
FAs['prob5_5_3'] = (\
  "$(a+b)(a+b)(a+b)(a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) == 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_5_3'])

## Problem 5.6:

---
Build an FA that accepts only those words that do *not* end with *ba*.

**Answer 5.6:**
RE: $Λ+a+(a+b)^{*}(b+aa)$

In [ ]:
FAs['prob5_6'] = (\
  "$Λ+a+(a+b)^{*}(b+aa)$"
, ARBNO(σ('a') + σ('b')) + σ('ba') + RPOS(0) + FAIL()
, [ [  ' ' , 'a', 'b']
  , ['-+1' ,  1 ,  2 ]
  , [ '+2' ,  3 ,  2 ]
  , [  '3' ,  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_6'])

## Problem 5.7:

---
Build an FA that accepts only those words that begin or end with a double letter.

**Answer 5.7:** RE: $\quad (aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$

In [ ]:
FAs['prob5_7'] = (\
  "$(aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$"
, ( POS(0) + (σ('aa') | σ('bb')) + ARB()
  | (σ('aa') | σ('bb')) + ARB() + RPOS(0)
  )
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  4 ]
  , [   '2',  3 ,  6 ]
  , [  '+3',  3 ,  3 ]
  , [   '4',  7 ,  5 ]
  , [  '+5',  5 ,  5 ]
  , [   '6',  7 ,  8 ]
  , [   '7',  9 ,  6 ]
  , [  '+8',  7 ,  8 ]
  , [  '+9',  9 ,  7 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_7'])

## Problem 5.8:

---
Build an FA that accepts only those words that have an even number of substrings *ab*.

**Answer 5.8: ???**
- RE: $\quad (abab)^{*}$
- RE: $\quad (aa^{*}bb^{*}aa^{*}bb^{*})^{*}$

In [ ]:
FAs['prob5_8_1'] = (\
  "$(abab)^{*}$"
, ARBNO(σ('abab'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  6 ]
  , [  '2',  6 ,  3 ]
  , [  '3',  4 ,  6 ]
  , [  '4',  6 ,  5 ]
  , [ '+5',  2 ,  6 ]
  , [  '6',  6 ,  6 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_8_1'])

In [ ]:
FAs['prob5_8_2'] = (\
  "$(aa^{*}bb^{*}aa^{*}bb^{*})^{*}$"
, ARBNO(
    σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  + σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  )
, [ [   ' ', 'a', 'b']
  , [ '-+1',  2 ,  1 ]
  , [   '2',  2 ,  3 ]
  , [   '3',  4 ,  3 ]
  , [   '4',  4 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_8_2'])

## Problem 5.9:

---
* (i) Recall from Chapter 4 the language of all words over the alphabet {*a b*} that have both the letter *a* and the letter *b* in them, but not necessarily in that order. Build an FA that accepts this language.
* (ii) Build an FA that accepts the language of all words with only *a*'s or only *b*'s in them. Give a regular expression for this language.

**Answer 5.9:**
- (i) RE: $(a+b)^{*}(ab+ba)(a+b)^{*}$ or $\quad (aa^{*}b+bb^{*}a)(a+b)^{*}$
- (ii) RE: $aa^{*}+bb^{*}$

In [ ]:
FAs['prob5_9_1'] = (\
  "$(a+b)^{*}(ab+ba)(a+b)^{*}$"
, ARB() + σ('a') + ARB() + σ('b') + ARB()
| ARB() + σ('b') + ARB() + σ('a') + ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [ '+4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_9_1'])

In [ ]:
FAs['prob5_9_2'] = (\
  "$aa^{*}+bb^{*}$"
, SPAN("a") | SPAN("b")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [ '+2',  2 ,  4 ]
  , [ '+3',  4 ,  3 ]
  , [  '4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_9_2'])

## Problem 5.10:

---
Consider all the possible FAs over the alphabet { a, b } that have exactly two states. An FA must have a designated start state, but there are four possible ways to place the *+*'s:

Type 1: ['-',  '']
Type 2: ['-+', '']
Type 3: ['-',  '+']
Type 4: ['-+', '+']

Each FA needs four edges (two from each state), each of which can lead to either of the states. There are $2^{4}=16$ ways to arrange the labeled edges for each of the four types of FAs. Therefore, in total there are 64 different FAs of two states. However, they do not represent 64 non-equivalent FAs because they are not all associated with different languages. All type 1 FAs do not accept any words at all, whereas all FAs of type 4 accepts all strings of *a*'s and *b*'s.

* (i) Draw the remaining FAs of type 2.
* (ii) Draw the remaining FAs of type 3.
* (iii) Recalculate the total number of two-state machines using the transition table definition.

**Book answer 5.10:**

* (i-ii) *See below*
* (iii) Transition tables for a two-state machine has four cells, two rows by two columns. Each may be filled with 1 or 2. So there are $2^4$ possibilities. There are still the same four ways to designate final states. Hence the total number of different transition tables is, surprise, 64, 2^4*4.

In [ ]:
FA_template = (\
  None
, ARB()
, [ [ ' ', 'a', 'b']
  , [ '1',  0 ,  0 ]
  , [ '2',  0 ,  0 ]])
#-------------------------------------------------------------------------------
for t in range(1,3):
    for i, combo in enumerate(itertools.product([1, 2], repeat=4)):
        name = f"prob_10_{i}"
        if t == 1:
            FA_template[2][1][0] = '-+1'
            FA_template[2][2][0] = '2'
        if t == 2:
            FA_template[2][1][0] = '-1'
            FA_template[2][2][0] = '+2'
        FA_template[2][1][1] = combo[0]
        FA_template[2][1][2] = combo[1]
        FA_template[2][2][1] = combo[2]
        FA_template[2][2][2] = combo[3]
        FAs[name] = FA_template
        display_fa_problem(FAs[name])

## Problem 5.11:

---
Show that there are exactly 5832 different finite automata with three states *x* *y*, *z* over the alphabet {*a b*}, where *x* is always the start state.

**Answer 5.11:** $1*2^3*3^6=5832$

## Problem 5.12:

---
Suppose a particular FA, called FIN, has the property that it had only one final state that was not the start state. During the night, vandals come and switch the *+* sign with the *-* sign and reverse the direction of all the edges.
* (i) Show that the picture that results might not actually be an FA at all by giving an example.
* (ii) Suppose, however, that in a particular case what resulted was, in fact, a perfectly good FA. Let us call it NIF. Give an example of one such machine.
* (iii) What is the relationship between the language accepted by FIN and the language accepted by NIF as described in part (ii)? Why?
* (iv) One of the vandals told me that if in FIN the plus state and the minus state were the same state, then the language accepted by the machine could contain only palindromic words. Defeat this vandal by example.

**Answer 5.12:**
* (i) Any FA with a state having more than one incoming of the same character is problematic
* (ii) Ensuring one of each in and out for each character of every state
* (iii) The language accepted by NIF is the reverse of every word in the language accepted by FIN. For every path from one state to another in FIN, the exact reverse path exists in NIF.

In [ ]:
FAs['prob5_12_1'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  3 ]
  , [ '+3',  3 ,  3 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_12_1'])

In [ ]:
FAs['prob5_12_2'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [ '+3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_12_2'])

In [ ]:
FAs['prob5_12_3'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [  '3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_12_3'])

## Problem 5.13:

---
We define a removable state as a state such that if we erase the state itself and the edges that come out of it, what results is a perfectly good-looking FA.
* (i) Give an example of an FA that contains a removable state.
* (ii) Show that if we erase a removable state the language defined by the reduced FA is exactly the same as the language defined by the old FA.

**Answer 5.13:**

* (i) *See below*
* (ii) An FA only retains its properties if the state being removed could not actually be reached, that is a state is removable only if there are no edges coming into it. Clearly if the stae is never used it has no bearing on the language that the FA accepts.

In [ ]:
FAs['prob5_13_1'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  1 ,  1 ]
  , [ '+2',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_13_1'])

## Problem 5.14:

---
* (i) Build an FA that accepts the language of all strings of *a*'s and *b*'s such that the next to the last letter is an *a*.
* (ii) Build an FA that accepts the language of all strings of length 4 or more such that the next-to-last letter is equal to the second letter of the input string.

**Answer 5.14:** *See below*

In [ ]:
FAs['prob5_14_1'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  1 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  3 ,  4 ]
  , [ '+4',  3 ,  1 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_14_1'], verbose=True)

In [ ]:
FAs['prob5_14_2'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  7 ]
  , [  '3',  4 ,  3 ]
  , [  '4',  5 ,  6 ]
  , [ '+5',  5 ,  6 ]
  , [ '+6',  5 ,  3 ]
  , [  '7',  7 ,  8 ]
  , [  '8', 10 ,  9 ]
  , [ '+9', 10 ,  9 ]
  , ['+10',  7 ,  9 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_14_2'], verbose=True)

## Problem 5.15:

---
Build a machine that accepts all strings that have an even length that is not divisible by 6.

In [ ]:
FAs['prob5_15'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  7 ,  7 ]
  , [  '7',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_15'])

## Problem 5.16:

---
Build an FA such that when the labels *a* and *b* are swapped the new machine is different from the old one but equivalent (the language defined by these machines is the same).

**Answer 5.16:** *See below*

In [ ]:
FAs['prob5_16'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  3 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [  '4',  3 ,  3 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_16'])

## Problem 5.17:

---
* (i-iii) Describe in English the languages accepted by the following FAs.
* (iv) Write regular expressions for the languages accepted by these three machines.

**Answer 5.17:**
* (i) odd-length sequence ending in *a*
* (ii) non-null even-length sequence ending in *a*
* (iii) non-null even-length sequence of *xa*
* (iv) *See below*

In [ ]:
FAs['prob5_17_1'] = ("$(aa+ab+ba+bb)^{*}a$", ARB, \
  [ [  ' ', 'a', 'b']
  , [  '1',  2 ,  2 ]
  , [ '-2',  3 ,  1 ]
  , [ '+3',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_17_1'])

In [ ]:
FAs['prob5_17_2'] = ("$(a+b)(aa+ab+ba+bb)a$", ARB,
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  2 ,  2 ]
  , [  '4',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_17_2'])

In [ ]:
FAs['prob5_17_3'] = ("$(aa+ba)(aa+ba)^{*}$", ARB,
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  5 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  3 ,  5 ]
  , [  '5',  5 ,  5 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_17_3'])

## Problem 5.18:

---
The following is an FA over the alphabet $\Sigma = \{ a\, b\, c \}$. Prove that it acepts all strings that have an odd number of ocurrences of the substring *abc*.

**Answer 5.18:** The machine will stay in states 1, 2, or 3 until *abc* is seen leading to state 4. The machine will stay in states 4, 5, or 6 until another *abc* is seen leading to state 1, and repeat. So no occurence of *abc* seen while in states 1-3, and one occurrence seen while in states 4-6. Due to repitition, this translates to even and odd. All states 4-6 are final states.

In [ ]:
FAs['prob5_18'] = ("", ARB,
  [ [  ' ', 'a', 'b', 'c']
  , [ '-1',  2 ,  1 ,  1 ]
  , [  '2',  2 ,  3 ,  1 ]
  , [  '3',  2 ,  1 ,  4 ]
  , [ '+4',  5 ,  4 ,  4 ]
  , [ '+5',  5 ,  6 ,  4 ]
  , [ '+6',  5 ,  4 ,  1 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_18'])

## Problem 5.19:

---
Consider the following FA:
* (i) Show that any input string with more than three letters is not accepted by this FA.
* (ii) Show that the only words accepted are *a*, *aab*, and *bab*.
* (iii) Show that by changing the location of *+* signs, we can make this FA accept the language { *bb* *aba* *bba* }.
* (iv) Show that any language in which the words have fewer than four letters can be accepted by a machine that looks like this one with the *+* signs in different places.
* (v) Prove that if L is a finite language, then there is some FA that accepts L extending the binary-tree part of this machine several more layers if necessary.

**Book answer 5.19:**
* (i) This FA starts out like a binary tree. For the first three letters of the input, each time we read a letter we reach some node on the next level. However reading any more letters traps us in the dead end state at the bottom. So no word of more than three letters can be accepted.
* (ii) There are exactly three final states in the machine. Concatenation of the labels on the transitions to these states shows that the machine accepts the words *a*, *aab*, and *bab*.
* (iii) Eliminate the three existing pluses. Place new pluses in each of the following states, the right-most node on level two, the third node from the left on level three, and the second node from the right also on level three.
* (iv) Each of the 15 possible strings with less than four letters is represented by one of the nodes (excluding the dead end state). Note the lexicographic order. Simply mark those nodes that represent the words in the language with plus signs.
* (v) Since L is finite, we can determine the longest word and then build a binary tree large enough to accommodate it. Next we add a dead end state with transitions from all the leaves and itself. For every word in L, we place a plus in the appropriate, unique state.

In [ ]:
FAs['prob5_19'] = ("", ARB,
  [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  3 ]
  , [   '2',  4 ,  5 ]
  , [   '3',  6 ,  7 ]
  , [   '4',  8 ,  9 ]
  , [   '5', 10 , 11 ]
  , [   '6', 12 , 13 ]
  , [   '7', 14 , 15 ]
  , [   '8', 16 , 16 ]
  , [   '9', 16 , 16 ]
  , [  '10', 16 , 16 ]
  , [  '11', 16 , 16 ]
  , [  '12', 16 , 16 ]
  , [  '13', 16 , 16 ]
  , [  '14', 16 , 16 ]
  , [  '15', 16 , 16 ]
  , [  '16', 16 , 16 ]])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['prob5_19'])

## Problem 5.20:

---
Let us consider the possibility of an infinite automaton that starts with this infinite binary tree. Let L be any infinite language of strings of *a*'s and *b*s whatsoever. Show that by the judicious placement of *+*'s, we can turn the picture above into an infinite automaton to accept the language L. Show that for any given finite string, we can determine from this machine, in a finite time, whether it is a word in L. Discuss why this machine would not be a satisfactory language-definer for L.

**Book answer 5.20:** Even though the binary tree is infinite, the machine retains the uniqueness property (of the machines described in problem 19) that the path for every string ends in a different state. Surely just by placing the plus signs in the nodes that represent words in L would constitute an infinite automata. However, the machine obviously fails to define the language because no one can write down the infintely large picture.

# **Chapter 6: Transition Graphs**

## Functions

### class TG:

In [ ]:
class TG:
    def __init__(self, Sigma, definition):
        self.Σ = set(Sigma) # input alphabet
        self.Q = set() # states
        self.S = set() # start states
        self.F = set() # final states
        self.Δ = set() # transitions (p, w, q)
        for name, edges in definition.items():
            state = name.lstrip('+-')
            self.Q.add(state)
            if '-' in name: self.S.add(state)
            if '+' in name: self.F.add(state)
            for label, target in edges:
                self.Q.add(str(target))
                self.Δ.add((state, label, str(target)))
        if not self.S: raise ValueError("TG must have at least one start state")

    def __repr__(self):
        return (f"TG(Q={self.Q}, Σ={self.Σ}, S={self.S}, F={self.F}, Δ={self.Δ}")

    def display(self):
        A = AGraph(directed=True, bgcolor="black", fontcolor="white", rankdir="LR")
        attributes = dict(
            shape='circle', color='white', fontcolor='white',
            style='filled', fillcolor='black', fixedsize="true")
        for state in self.Q:
            if state in self.F:
                  A.add_node(state, label=state, width="0.35", height="0.35", peripheries='2', **attributes)
            else: A.add_node(state, label=state, width="0.50", height="0.50", **attributes)
    #      if state in self.F: # white filled
    #            A.add_node(state, label=state, shape='doublecircle',
    #                color='white', style='filled', fillcolor='white', fontcolor='black', penwidth='2')
    #      else: A.add_node(state, label=state, shape='circle',
    #                color='white', style='filled', fillcolor='white', fontcolor='black')
        for source in self.Q:
            for target in self.Q:
                labels = [label for s, label, t in sorted(self.Δ) if s == source and t == target]
                labels = ", ".join(labels)
                if labels != "":
                    A.add_edge(source, target, label=labels, fontcolor="white", color="white")
        for state in self.S:
            A.add_node(f"start_{state}", shape="point", style="invis")
            A.add_edge(f"start_{state}", state, color="white")
        A.layout(prog="dot")
        display(SVG(data=A.draw(format="svg")))

### run_tg()

In [ ]:
def run_tg(TG, string):
    def STATE(state): return int(state.replace('-', '').replace('+', ''))
    start = {STATE(state) for state, _ in TG if '-' in state}
    final = {STATE(state) for state, _ in TG if '+' in state}
    agenda = {(s, string) for s in start}
    visited = set()
    while agenda:
        next_agenda = set()
        for state, remaining in agenda:
            if (state, remaining) not in visited:
                visited.add((state, remaining))
                if state in final and remaining == "":
                    return True
                for target, label in TG[state-1][1]:
                    if label == "Λ":
                        next_agenda.add((target, remaining))
                    elif remaining.startswith(label):
                        next_agenda.add((target, remaining[len(label):]))
        agenda = next_agenda
    return False

## Problem 6.1:

---
For each of the five FAs pictured in Problems 17, 19, and 20 in Chapter 5, build a transition graph that accepts the same language but has fewer states.

**Answer 6.1:**

## Problem 6.2:

---
For each of the next 10 words, decide which of the six machines on the next page accept the given word.
* (i) Λ
* (ii) *a*
* (iii) *b*
* (iv) *aa*
* (v) *ab*
* (vi) *aba*
* (vii) *abba*
* (viii) *bab*
* (ix) *baab*
* (x) *abbb*

**Answer 6.2:**

In [ ]:
TGs['prob6_2_1'] = TG(\
"ab",
{  '-1': [("a", 2), ("ab", 4), ("b", 4)]
,   '2': [("b", 2), ("Λ", 3)]
,   '3': [("b", 3), ("a", 4)]
,  '+4': []
})
#-------------------------------------------------------------------------------
TGs['prob6_2_1'].display()

In [ ]:
TGs['prob6_2_2'] = TG(\
"ab",
{ '-+1': [("a", 2), ("b", 3)]
,   '2': [("a", 2), ("b", 3)]
,  '+3': []
})
#-------------------------------------------------------------------------------
TGs['prob6_2_2'].display()

In [ ]:
TGs['prob6_2_3'] = TG(\
"ab",
{  '-1': [("ab", 1), ("ba", 1), ("aa", 2), ("bb", 2)]
,  '+2': [("ab", 2), ("ba", 2), ("aa", 2), ("bb", 2)]
})
#-------------------------------------------------------------------------------
TGs['prob6_2_3'].display()

In [ ]:
TGs['prob6_2_4'] = TG(\
"ab",
{  '-1': [("a", 2), ("b", 3)]
,  '+2': [("a", 1), ("b", 2)]
,  '+3': [("a", 3), ("b", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob6_2_4'].display()

In [ ]:
TGs['prob6_2_5'] = TG(\
"ab",
{ '-+1': [("Λ", 2), ("Λ", 5)]
,   '2': [("a", 3), ("b", 2)]
,   '3': [("a", 4), ("b", 3)]
,  '+4': [("b", 4)]
,   '5': [("a", 5), ("b", 6)]
,   '6': [("a", 6), ("b", 7)]
,  '+7': [("a", 7)]
})
#-------------------------------------------------------------------------------
TGs['prob6_2_5'].display()

In [ ]:
TGs['prob6_2_6'] = TG(\
"ab",
{  '-1': [("ab", 2), ("a", 4)]
,  '+2': [("bb", 3)]
,  '+3': []
,   '4': [("b", 4), ("a", 3)]
})
#-------------------------------------------------------------------------------
TGs['prob6_2_6'].display()

## Problem 6.3:

---
Show that any language that can be accepted by a TG can be accepted by a TG with an even number of states.

**Answer 6.3:**

## Problem 6.4:

---
How many different TGs are there over the alphabet { *a*, *b* } that have two states?

**Answer 6.4:**

## Problem 6.5:

---
Prove that for every TG there is another TG that accepts the same language but has only one *+* state.

**Answer 6.5:**

## Problem 6.6:

---
Build a TG that accepts the language $L_1$ of all words that begin and end with the same double letter, either of the form *aa...aa* or *bb...bb*. Note: *aaa* and *bbb* are not words in this language.

**Answer 6.6:**

## Problem 6.7:

---
If OURSPONSOR is a language that is accepted by a TG called Henry, prove that there is a TG that accepts the language of all strings of *a*'s and *b*'s that end in a word from OURSPONSOR.

**Answer 6.7:**

## Problem 6.8:

---
* (i) Suppose that $L$ is a finite language whose words are $w_1, w_2, w_3, ..., w_{83}$. Prove that there is a TG that accepts exactly the language $L$.
* (ii) Of all TGs that accept exactly the language $L$, what is the one with the fewest number of states?

**Answer 6.8:**

## Problem 6.9:

---
Given a TG, called $TG_1$, that accepts the language $L_1$ and a TG, called $TG_2$, that accepts the language $L_2$, show how to build a new TG (called $TG_3$) that accepts exactly the language $L_1 + L_2$.

**Answer 6.9:**

## Problem 6.10:

---
Given $TG_1$ and $TG_2$ as described in Problem 6.9, show how to build $TG_4$ that accepts exactly the language $L_1L_2.$

**Answer 6.10:**

## Problem 6.11:

---
Given a TG for some arbitrary language $L$, what language would it accept if every *+* state were to be connected back to every *-* state by Λ-edges? For example, by the following method:

*Hint*: Why is the answer not always $L^{*}$.

**Answer 6.11:**

In [ ]:
TGs['prob6_11_1'] = TG(\
"ab",
{  '-1': [("a", 2), ("bb", 3)]
,   '2': [("ba", 2), ("a", 3), ("b", 4)]
,  '+3': [("ab", 4)]
,  '+4': [("a", 4)]
})
#-------------------------------------------------------------------------------
TGs['prob6_11_1'].display()

In [ ]:
TGs['prob6_11_2'] = TG(\
"ab",
{  '-1': [("a", 2), ("bb", 3)]
,   '2': [("ba", 2), ("a", 3), ("b", 4)]
,  '+3': [("ab", 4), ("Λ", 1)]
,  '+4': [("a", 4), ("Λ", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob6_11_2'].display()

## Problem 6.12:

---
* (i) Let the language $L$ be accepted by the transition graph $T$ and let $L$ not contain the word $Λ$. Show how to build a new TG that accepts exactly all the words in $L$ and the word $Λ$.
* (ii) Given $TG_1$, that accepts the language $L_1$, show how to build a TG that accepts the language $L^{*}$. *Hint*: Use Problem 11 and 12(i) and sound authoritative.

**Answer 6.12:**

## Problem 6.13:

---
Using the results of Problems 8, 9, 10, and 12 in an organized fashion, prove that if $L$ is any language that can be defined by a regular expression, then there is a $TG$ that accepts exactly that language $L^{*}$.

**Answer 6.13:**

## Problem 6.14:

---
Verify that there are indeed three and only three ways for the $TG$ on p. 84 to accept the word *abbbabbbabba*.

**Answer 6.14:**

## Problem 6.15:

---
An FA with four states was sitting unguarded one night when vandals came and stole an edge labeled *a*. What resulted was a TG that accepted exactly the language $b^{*}$. In the morning the FA was repaired, but the next night vandals stole an edge labeled *b* and what resulted was a TG that accepted $a^{*}$. The FA was again repaired, but this time the vandals stole two edges, one labeled *a* and one labeled *b*, and the resulting TG accepted the language $a^{*}+b^{*}$. What was the original FA?

**Answer 6.15:**

## Problem 6.16:

---
Let the language $L$ be accepted by the transition graph $T$ and let $L$ not contain the word *ba*. We want to build a new TG that accepts exactly $L$ and the word *ba*.
* (i) One suggestion is to draw an edge from *-* to *+* and label it *ba*. Show that this does not work.
* (ii) Another suggestion is to draw a new *+* state and draw an edge from any *-* state to it labeled *ba*. Show that this does not always work.
* (iii) What does work?

**Answer 6.16:**

## Problem 6.17:

---
Let $L$ be any language. Let us define the **transpose** of $L$ to be the language of exactly those words in $L$ spelled backward. If $w \in L$, then $reverse(w) \in L$. For example, if $$L= \{ a\, abb\, bbaab\, bbbaa\}$$ then $$transpose(L)=\{ a\, bba\, baabb\, aabbb\}$$

* (i) Prove that there is an FA that accepts $L$, then there is a TG that accepts the transpose of $L$.
* (ii) Prove that if there is a TG that accepts $L$, then there is a TG that accepts the transpose of $L$. *Note*: It is true, but much harder to prove, that if an FA accepts $L$, then some FA accepts the transpose of $L$. However, after Chapter 7 this will be trivial to prove.
* (iii) Prove that transpose($L_1L_2$) = (transpose($L_2$))(transpose($L_1$)).

**Answer 6.17:**

## Problem 6.18:

---
Transition graph $T$ accepts langauge $L$. Show that if $L$ has a word of odd length, then $T$ has an edge with a label with an odd number of letters.

**Answer 6.18:**

## Problem 6.19:

---
A student walks into a classroom and sees on the blackboard a diagram of a TG with two states that accepts only the word Λ. The student reverses the direction of exactly one edge, leaving all other edges and all labels and all *+*'s and *-*'s the same. But now the new TG accepts the language $a^{*}$. What was the original machine?

**Answer 6.19:**

## Problem 6.20:

---
Let us now consider an algorithm for determining whether a specific TG that has no Λ-edges accepts a given word:
* Step 1: Number each edge in the TG in any order with the integers $1,2,3,...,x$, where *x* is the number of edges in the TG.
* Step 2: Observe that if the word has *y* letters and is accepted at all by this machine, it can be accepted by tracing a path of not more than *y* edges.
* Step 3: List all strings of *y* or fewer integers, each of which $\leq x$. This is a finite list.
* Step 4: Check each string on the list in step 3 by concatenating the labels of the edges involved to see whether they make a path from *-* to *+* corresponding to the given word.
* Step 5: If there is a string in step 4 that works, the word is accepted. If none work, the word is not in the language of the machine.

Challenges/Questions:
* (i) Prove that this algorithm does the job.
* (ii) Why is it necessary to assume that the TG has no Λ-edges?

**Answer 6.20:**

# **Chapter 7: Kleene's Theorem**

## Functions

### normalize_state_name

In [ ]:
def normalize_state_name(state):
    return int(state.replace('-', '').replace('+', ''))

### regular expression tree builders

In [ ]:
#-------------------------------------------------------------------------------
def literal(s): return ['σ', s]
def epsilon(): return ['Λ']
def kleene_star(t): return ['ς', t, '*']
#-------------------------------------------------------------------------------
def concatenate(*args):
    parts = []
    for part in args:
        assert part is not None
        if part == []: continue
        if part == ['Λ']: continue
        if part[0] == 'Σ':
            parts.extend(part[1:])
        else: parts.append(part)
    if len(parts) == 0: return epsilon()
    elif len(parts) == 1: return parts[0]
    else: return ['Σ'] + parts
#-------------------------------------------------------------------------------
def alternate(*args):
    parts = []
    for part in args:
        assert part is not None
        if part[0] == 'Π':
            parts.extend(part[1:])
        else: parts.append(part)
    assert len(parts) > 0 # return ['∅']
    if len(parts) == 1:
        return parts[0]
    else: return ['Π'] + parts

### convert_tg2re() [using NetworkX]

In [ ]:
#-------------------------------------------------------------------------------
def convert_tg2re(TG):
    edges = {}
    start_states, final_states = set(), set()
    for code, out_edges in TG.items():
        state = normalize_state_name(code)
        if '-' in code: start_states.add(state)
        if '+' in code: final_states.add(state)
        edges[state] = out_edges

    G = nx.DiGraph()
    for s, out_edges in edges.items():
        for t, label in out_edges:
            tree = epsilon() if label == 'Λ' else literal(label)
            G.add_edge(s, t, label=tree)
#   Step 1 ---------------------------------------------------------------------
    start, final = 0, max(edges.keys()) + 1
    G.add_nodes_from([start, final])
    for s in start_states: G.add_edge(start, s, label=epsilon())
    for f in final_states: G.add_edge(f, final, label=epsilon())
    matched = True
    while matched:
    #   Step 3 -----------------------------------------------------------------
        for p, q in list(G.edges()):
            edge_data = G.get_edge_data(p, q)
            assert edge_data
            if len(edge_data) > 1:
                labels = [value['label'] for value in edge_data.values()]
                G.remove_edges_from([(p, q, key) for key in list(edge_data.keys())])
                G.add_edge(p, q, label=alternate(*labels))
                matched = True
    #   Step 2(a) --------------------------------------------------------------
        P = rx.PyDiGraph()
        P.add_nodes_from(['p', 'q', 'r'])
        P.add_edges_from([('p','q'), ('q','q'), ('q','r')])
        GM = nx.DiGraphMatcher(G, P, node_match=lambda *_: True, edge_match=lambda *_: True)
        for m in GM.subgraph_isomorphisms_iter():
            p, q, r = m['p'], m['q'], m['r']
            if q in (start, final): continue
            in_label   = [d['label'] for s, _, d in G.in_edges(q, data=True) if s == p][0]
            out_label  = [d['label'] for _, t, d in G.out_edges(q, data=True) if t == r][0]
            loop_label = [d['label'] for s, t, d in G.edges(q, data=True) if s == q and t == q][0]
            loop_label = kleene_star(loop_label)
            G.add_edge(p, r, label=concatenate(in_label, loop_label, out_label))
            G.remove_node(q)
            matched = True
    #   Step 2(b) --------------------------------------------------------------
        P = rx.PyDiGraph()
        P.add_nodes_from(['p', 'q', 'r'])
        P.add_edges_from([('p','q'), ('q','r')])
        GM = nx.DiGraphMatcher(G, P, node_match=lambda *_: True, edge_match=lambda *_: True)
        for m in GM.subgraph_isomorphisms_iter():
            p, q, r = m['p'], m['q'], m['r']
            if q in (start, final): continue
            in_label   = [d['label'] for s, _, d in G.in_edges(q, data=True) if s == p][0]
            out_label  = [d['label'] for _, t, d in G.out_edges(q, data=True) if t == r][0]
            G.add_edge(p, r, label=concatenate(in_label, out_label))
            G.remove_node(q)
            matched = True
#   Step 4: --------------------------------------------------------------------
    data = G.get_edge_data(start, final)
    if data:
        return [value['label'] for value in data.values()][0]
    else: return None

### class DirectedGraph

In [ ]:
class DirectedGraph:
    def __init__(self):
        self._G = rx.PyDiGraph()
        self._state_to_idx = {}
        self._idx_to_state = {}

    def add_node(self, state_id):
        if state_id in self._state_to_idx:
            return self._state_to_idx[state_id]
        idx = self._G.add_node(state_id)
        self._state_to_idx[state_id] = idx
        self._idx_to_state[idx] = state_id
        return idx

    def add_nodes_from(self, state_ids):
        for s in state_ids:
            self.add_node(s)

    def remove_node(self, state_id):
        idx = self._state_to_idx.pop(state_id)
        self._idx_to_state.pop(idx)
        self._G.remove_node(idx)

    def node_indices(self):
        return list(self._state_to_idx.values())

    def nodes(self):
        return list(self._state_to_idx.keys())

    def has_edge(self, u_state, v_state):
        u_idx = self._state_to_idx[u_state]
        v_idx = self._state_to_idx[v_state]
        self._G.has_edge(u_idx, v_idx)

    def add_edge(self, u_state, v_state, payload):
        u_idx = self._state_to_idx[u_state]
        v_idx = self._state_to_idx[v_state]
        self._G.add_edge(u_idx, v_idx, payload)

    def get_edge_data(self, u_state, v_state):
        u_idx = self._state_to_idx.get(u_state)
        v_idx = self._state_to_idx.get(v_state)
        if u_idx is None or v_idx is None:
            return None
        return self._G.get_edge_data(u_idx, v_idx)

    def in_edges(self, state_id):
        idx = self._state_to_idx[state_id]
        for u_idx, _ in self._G.in_edges(idx):
            yield self._idx_to_state[u_idx], state_id

    def out_edges(self, state_id):
        idx = self._state_to_idx[state_id]
        for v_idx, _ in self._G.out_edges(idx):
            yield state_id, self._idx_to_state[v_idx]

    def edges(self, state_id=None):
        if state_id is None:
            for u_idx, v_idx, data in self._G.weighted_edge_list():
                yield self._idx_to_state[u_idx], self._idx_to_state[v_idx], data
        else:
            idx = self._state_to_idx[state_id]
            for u_idx, v_idx, data in self._G.weighted_edge_list():
                if u_idx == idx or v_idx == idx:
                    yield self._idx_to_state[u_idx], self._idx_to_state[v_idx], data

    def edge_list(self):
        for u_idx, v_idx in self._G.edge_list():
            yield self._idx_to_state[u_idx], self._idx_to_state[v_idx]

    def weighted_edge_list(self):
        for u_idx, v_idx, data in self._G.weighted_edge_list():
            yield self._idx_to_state[u_idx], self._idx_to_state[v_idx], data

    def remove_edge(self, u_state, v_state):
        u_idx = self._state_to_idx[u_state]
        v_idx = self._state_to_idx[v_state]
        self._G.remove_edge(u_idx, v_idx)

    def clear_edges(self):
        self._G.clear_edges()

    def vf2_mapping(self, pattern, node_matcher=None, edge_matcher=None):
        return rx.vf2_mapping(self._G, pattern._G,
                              node_matcher=node_matcher,
                              edge_matcher=edge_matcher)

    def __len__(self):
        return len(self._state_to_idx)

    def __contains__(self, state_id):
        return state_id in self._state_to_idx

### add_edge_with_tree

In [ ]:
def add_edge_with_tree(G, u, v, tree):
    if G.has_edge(u, v):
        payload = G.get_edge_data(u, v)
        payload.append(tree)
    else: G.add_edge(u, v, [tree])

### convert_tg2re()

In [ ]:
def convert_tg2re(TG):
    edges = {}
    start_states, final_states = set(), set()
    for code, out_edges in TG.items():
        state = normalize_state_name(code)
        if '-' in code: start_states.add(state)
        if '+' in code: final_states.add(state)
        edges[state] = out_edges

    G = DirectedGraph()
    for s in sorted(edges.keys()):
        G.add_node(s)
    for s, out_edges in edges.items():
        for t, label in out_edges:
            tree = epsilon() if label == 'Λ' else literal(label)
            add_edge_with_tree(G, s, t, tree)

#   Step 1
    start, final = 0, max(edges.keys()) + 1
    G.add_node(start)
    G.add_node(final)
    for s in start_states: add_edge_with_tree(G, start, s, epsilon())
    for f in final_states: add_edge_with_tree(G, f, final, epsilon())

    matched = True
    while matched:
    #   Step 3
        for u, v, payload in G.weighted_edge_list():
            if payload and len(payload) > 1:
                expr = alternate(*payload)
                payload.clear()
                payload.add(expr)
                matched = True
    #   Step 2(a)
        P = DirectedGraph()
        p_pat, q_pat, r_pat = P.add_node("p"), P.add_node("q"), P.add_node("r")
        P.add_edge(p_pat, q_pat, None)
        P.add_edge(q_pat, q_pat, None)
        P.add_edge(q_pat, r_pat, None)
        for mapping in G.vf2_mapping(P, node_matcher=lambda _, __: True, edge_matcher=lambda _, __: True):
            p, q, r = mapping[p_pat], mapping[q_pat], mapping[r_pat]
            if q in (start, final): continue
            in_label   = G.get_edge_data(p, q)[0]
            out_label  = G.get_edge_data(q, r)[0]
            loop_label = G.get_edge_data(q, q)[0]
            loop_expr  = kleene_star(loop_label)
            G.remove_edge(p, q)
            G.remove_edge(q, r)
            G.remove_edge(q, q)
            add_edge_with_tree(G, p, r, concatenate(in_label, loop_expr, out_label))
            if (G.degree() == 0): G.remove_node(q)
            matched = True
    #   Step 2(b)
        P.remove_edge(q, q)
        for mapping in G.vf2_mapping(P, node_matcher=lambda _, __: True, edge_matcher=lambda _, __: True):
            p, q, r = mapping[p_pat], mapping[q_pat], mapping[r_pat]
            if q in (start, final): continue
            in_payload  = G.get_edge_data(p, q)
            out_payload = G.get_edge_data(q, r)
            for in_label in in_payload:
                for out_label in out_payload:
                    new_expr = concatenate(in_label, out_label)
                    add_edge_with_tree(G, p, r, new_expr)
            if (G.degree() == 0): G.remove_node(q)
            matched = True
#   Step 4
    return G.get_edge_data(start, final)

## Problem 7.1

---
Using the bypass algorithm in the proof of Theorem 6, Part 2, convert each of the following TGs into regular expressions:

(i-vi) *See below*

In [ ]:
# Answer 7.1
TGs['prob7_1_1'] = TG(\
"ab",
{  '-1': [("a", 1), ("b", 1), ("Λ", 2)]
,   '2': [("b", 3)]
,  '+3': []
})
#-------------------------------------------------------------------------------
TGs['prob7_1_1'].display()

In [ ]:
TGs['prob7_1_2'] = TG(\
"ab",
{  '-1': [("ab", 2), ("bb", 4)]
,   '2': [("a", 3)]
,  '+3': [("a", 3), ("b", 3)]
,   '4': [("a", 3)]
})
#-------------------------------------------------------------------------------
TGs['prob7_1_2'].display()

In [ ]:
TGs['prob7_1_3'] = TG(\
"ab",
{  '-1': [("a", 1), ("b", 1), ("a", 2), ("b", 3), ("ab", 4)]
,  '+2': []
,  '+3': []
,   '4': [("ab", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob7_1_3'].display()

In [ ]:
TGs['prob7_1_4'] = TG(\
"ab",
{ '-+1': [("a", 1), ("b", 1), ("baa", 2)]
,  '+2': [("a", 2), ("b", 2), ("abb", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob7_1_4'].display()

In [ ]:
TGs['prob7_1_5'] = TG(\
"ab",
{  '-1': [("a", 1), ("ab", 2), ("abb", 4), ("bb", 5)]
,   '2': [("ba", 3)]
,   '3': [("Λ", 4)]
,  '+4': [("a", 4), ("b", 4)]
,   '5': [("Λ", 4)]
})
#-------------------------------------------------------------------------------
TGs['prob7_1_5'].display()

In [ ]:
TGs['prob7_1_6'] = TG(\
"ab",
{  '-1': [("ab", 2)]
,   '2': [("a", 2), ("ba", 3), ("ab", 4)]
,   '3': [("a", 1)]
,  '+4': [("bb", 5)]
,   '5': [("b", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob7_1_6'].display()

## Problem 7.2

---
In Chapter 5, Problem 10, we began the discussion of all possible FAs with two states. Write a regular expression for each machine of type 2 and type 3 by using the conversion algorithm described in the proof of Theorem 6, Part 2. Even though there is no algorithm for recognizing the languages, try to identify as many as possible in the attempt to discover how many different languages can be accepted by a two-state FA.

**Answer 7.2**

## **For problems 3 through 12**

use the following machines:

In [ ]:
#-------------------------------------------------------------------------------
FAs['FA71'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  1]
  , [ '+2',  2 ,  1]
  ])
#-------------------------------------------------------------------------------
FAs['FA72'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  1]
  , [  '2',  2 ,  3]
  , [ '+3',  3 ,  3]
  ])
#-------------------------------------------------------------------------------
FAs['FA73'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3]
  , [  '2',  4 ,  3]
  , [  '3',  2 ,  4]
  , [ '+4',  4 ,  4]
  ])
#-------------------------------------------------------------------------------
FAs['FA74'] = ("", ARB(), \
  [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  1]
  , [  '2',  2 ,  2]
  ])
#-------------------------------------------------------------------------------
display_fa_problem(FAs['FA71'], 4, False)
display_fa_problem(FAs['FA72'], 5, False)
display_fa_problem(FAs['FA73'], 5, False)
display_fa_problem(FAs['FA74'], 4, False)

## Problem 7.3

---
Using the algorithm of Kleene's theorem, Part 3, Rule 2, Proof 1, construct FAs for the following union languages:

* (i) $FA_1+FA_2$
* (ii) $FA_1+FA_3$
* (iii) $FA_2+FA_3$

**Answer 7.3**

## Problem 7.4

---
Using the algorithm of Kleene's theorem, Part 3, Rule 2, Proof 2, construct NFAs for the following languages:

* (i) $FA_1+FA_2$
* (ii) $FA_1+FA_3$
* (iii) $FA_2+FA_3$

**Answer 7.4**

## Problem 7.5

---
Using the algorithm of Theorem 6, Part3, Rule 3, construct FAs for the following product languages:

* (i) $FA_1FA_2$
* (ii) $FA_1FA_3$
* (iii) $FA_1FA_1$
* (iv) $FA_2FA_1$
* (v) $FA_2FA_2$

**Answer 7.5**

## Problem 7.6

---
Using the algorithm of Part 3, Rule 4, construct FAs for the following languages:

* (i) $(FA_1)^{*}$
* (ii) $(FA_2)^{*}$

**Answer 7.6**

## Problem 7.7

---
We are now interested in proving Part 3, Rule 3, of Kleene's theorem by NFAs. The basic theory is that when we reach any *+* state in $FA_1$, we could continue to $FA_1$ by following its *a*-edge and *b*-edge, or we could pretend that we have jumped to $FA_2$ by following the *a*-edge and *b*-edge coming out of the start state on $FA_2$.

We do not change any states or edges in either machine; we merely add some new (nondeterministic) edges from *+* states in $FA_1$, to the destination states of $FA_2$'s start state. Finally, we erase the *+*'s from $FA_1$ and the *-* sign from $FA_2$, and we have the desired NFA.

Let us illustrate this by multiplying $FA_1$ and $FA_2$ above. Find NFAs for the following product languages:

* (i) $FA_1FA_1$
* (ii) $FA_1FA_3$
* (iii) $FA_2FA_2$

**Answer 7.7**

In [ ]:
TGs['prob7_7'] = TG(\
"ab",
{  '-1': [("b", 1), ("a", 2)]
,   '2': [("a", 2), ("b", 3), ("a", 4)]
,   '3': [("b", 3), ("a", 4)]
,   '4': [("a", 4), ("b", 5)]
,  '+5': [("a", 5), ("b", 5)]
})
#-------------------------------------------------------------------------------
TGs['prob7_7'].display()

## Problem 7.8

---
Take the three NFAs in Problem 7 above and convert them into FAs by the algorithm of Theorem 7.

**Answer 7.8**

## Problem 7.9

---
We can use NFAs to prove Theorem 6, Part 3, Rule 4, as well. The idea is to allow a nondeterministic jump from any *+* state to the states reachable from the *-* state by *a*-edges and *b* edges.

* (i)  Provide the details for this proof by constructive algorithm.
* (ii) Draw the resultant NFA for $(FA_1)^{*}$.
* (iii) Draw the resultant NFA for $(FA_2)^{*}$.
* (iv) Draw the resultant NFA for $(FA_3)^{*}$.

**Answer 7.9**

## Problem 7.10

---
Convert the machines in Problem 9(ii) and (iii) above to FAs by the algorithm in the proof of Theorem 7.

**Answer 7.10**

## Problem 7.11

---
Find FAs for the following languages:

* (i) $FA_4FA_4$
* (ii) $(FA_4)^{*}$

**Answer 7.11**

## Problem 7.12

---
* (i) Is the machine for $FA_1FA_1$ (Problem 5) the same as the machine for $(FA_1)^{*}$ (Problem 6)? Are the languages the same?
* (ii) Is the machine for $FA_4FA_4$ the same as the machine for $(FA_4)^{*}$ (Problem 11)? Are the languages the same?

**Answer 7.12**

## Problem 7.13

---
* (i) For the examples derived earlier, which algorithmic method produces product machines with fewer states, the direct (Problem 5) or the NFA (Problem 8)?
* (ii) If some automaton, $FA_1$, has *n* states and some other automaton, $FA_2$ has *m* states, what are the maximum number of states possible in each of the machines corresponding to $FA_1+FA_2,FA_1FA_2,(FA_1)^{*}$ that are produced.
  - (a) By the subset method described in the proof of Kleen's theorem.
  - (b) By building NFAs and then converting them into FAs.

**Answer 7.13**

## Problem 7.14

---
Convert each of the following NFAs into FAs using the constructive algorithm presented in Proof 2 of Theorem 7.

**Answer 7.14**

In [ ]:
TGs['prob7_14_1'] = TG(\
"ab",
{ '-+1': [("a", 1), ("b", 1), ("b", 2)]
,   '2': []
})
#-------------------------------------------------------------------------------
TGs['prob7_14_1'].display()

In [ ]:
TGs['prob7_14_2'] = TG(\
"ab",
{  '-1': [("a", 2), ("a", 3)]
,   '2': [("b", 3)]
,  '+3': []
})
#-------------------------------------------------------------------------------
TGs['prob7_14_2'].display()

In [ ]:
TGs['prob7_14_3'] = TG(\
"ab",
{  '-1': [("a", 2), ("b", 2)]
,   '2': [("a", 3)]
,  '+3': [("a", 2), ("b", 2)]
})
#-------------------------------------------------------------------------------
TGs['prob7_14_3'].display()

In [ ]:
TGs['prob7_14_4'] = TG(\
"ab",
{  '-1': [("a", 1), ("b", 2), ("b", 3)]
,   '2': [("b", 3)]
,  '+3': [("a", 3)]
})
#-------------------------------------------------------------------------------
TGs['prob7_14_4'].display()

In [ ]:
TGs['prob7_14_5'] = TG(\
"ab",
{  '-1': [("a", 2), ("b", 3)]
,  '+2': [("a", 1)]
,   '3': [("b", 2)]
})
#-------------------------------------------------------------------------------
TGs['prob7_14_5'].display()

In [ ]:
TGs['prob7_14_6'] = TG(\
"ab",
{  '-1': [("a", 2)]
,   '2': [("a", 2), ("b", 2), ("b", 3)]
,  '+3': []
})
#-------------------------------------------------------------------------------
TGs['prob7_14_6'].display()

In [ ]:
TGs['prob7_14_7'] = TG(\
"ab",
{  '-1': [("a", 2)]
,   '2': [("a", 2), ("b", 2), ("b", 3)]
,  '+3': [("b", 4)]
,  '+4': []
})
#-------------------------------------------------------------------------------
TGs['prob7_14_7'].display()

In [ ]:
TGs['prob7_14_8'] = TG(\
"ab",
{  '-1': [("a", 2)]
,   '2': [("a", 2), ("b", 2), ("b", 3)]
,   '3': [("b", 4)]
,  '+4': []
})
#-------------------------------------------------------------------------------
TGs['prob7_14_8'].display()

In [ ]:
TGs['prob7_14_9'] = TG(\
"ab",
{  '-1': [("a", 1), ("b", 1), ("a", 2), ("a", 3)]
,  '+2': []
,   '3': [("a", 2), ("b", 2)]
})
#-------------------------------------------------------------------------------
TGs['prob7_14_9'].display()

In [ ]:
TGs['prob7_14_10'] = TG(\
"ab",
{  '-1': [("a", 2), ("a", 3)]
,   '2': [("b", 3), ("a", 4)]
,   '3': [("a", 4)]
,  '+4': [("b", 1)]
})
#-------------------------------------------------------------------------------
TGs['prob7_14_10'].display()

## **For Problems 15 through 17**

let us now introduce a machine called **a nondeterministic finite automata with null string labels,** abbreviated NFA_Λ. This machine follows the same rules as an NFA except that we are allowed to have edges labeled $Λ$.

## Problem 7.15

---
Show that it possible to use a technique analagous to that used in Proof 2 of Theorem 7 to constructively convert an NFA_Λ into an FA by explicitly giving the steps of the conversion process.

**Answer 7.15**

## Problem 7.16

---
Convert the following NFA_Λ's into FAs using the algorithm invented in Problem 15:

* (i-iii) *See below*

**Answer 7.16**

In [ ]:
TGs['prob7_16_1'] = TG(\
"ab",
{  '-1': [("a", 2), ("Λ", 4), ("b", 4)]
,   '2': [("Λ", 3), ("a", 5)]
,  '+3': []
,  '-4': [("a", 2), ("b", 5)]
,   '5': [("a", 3), ("a", 5)]
})
#-------------------------------------------------------------------------------
TGs['prob7_16_1'].display()

In [ ]:
TGs['prob7_16_2'] = TG(\
"ab",
{  '-1': [("Λ", 2), ("Λ", 3)]
,   '2': [("a", 2), ("a", 3), ("Λ", 4)]
,   '3': [("Λ", 2), ("a", 4)]
,  '+4': []
})
#-------------------------------------------------------------------------------
TGs['prob7_16_2'].display()

In [ ]:
TGs['prob7_16_3'] = TG(\
"ab",
{  '-1': [("a", 2), ("Λ", 3)]
,   '2': [("Λ", 2), ("b", 3)]
,   '3': [("a", 3), ("a", 4)]
,  '+4': [("b", 1), ("b", 4)]
})
#-------------------------------------------------------------------------------
TGs['prob7_16_3'].display()

## Problem 7.17

---
Using the result in Problem 15, find a third proof of Part 3 of Keene's theorem:

* (i) Rule 2
* (ii) Rule 3
* (iii) Rule 4

**Answer 7.17**

## Problem 7.18

---
* (i) Find two different machines $FA_1$ and $FA_2$ such that the languages accepted by $FA_1+FA_2$ and $FA_1FA_2$ are the same, yet the machines generated by the algorithm in the proof of Theorem 6 are different.
* (ii) Find two different machines $FA_1$ and $FA_2$ such that the algorithm in the proof of Theorem 6 creates the same machine for $(FA_1)^{*}$ and $(FA_2)^{*}$.

**Answer 7.18**

## Problem 7.19

---
For the language accepted by the following machine, find a different FA with four states. Find an NFA that accepts the same language and has only seven edges (where edges with two labels are counted twice).

**Answer 7.19**

In [ ]:
TGs['prob7_19'] = TG(\
"ab",
{  '-1': [("b", 1), ("a", 2)]
,   '2': [("a", 3), ("b", 1)]
,   '3': [("a", 4), ("b", 5)]
,   '4': [("a", 4), ("b", 5)]
,  '+5': [("a", 4), ("b", 5)]
})
#-------------------------------------------------------------------------------
TGs['prob7_19'].display()

## Problem 7.20

---
A one-person game can be converted into an NFA as follows. Let every possible board situation be a state. If any move (there may be several types of moves, but we are not interested in distinguishing them) can change from state *x* into some state *y*, then draw an edge from *x* to *y* and label it *m*. Label the initial position *-* and the winning positions *+*. "This game can be won in five moves" is the same as saying, "$m^5$ is accepted by this NFA." Once we have the NFA, we use the alogorithm of Chapter 7 to convert it into a regular expression. The language it represents tells us how many moves are in each winning sequence.

Let us do this with the following example. The game of Flips is played with three coins. Initially, they are all heads. A move consists of flipping two coins simultaneously from what ever they were to the opposite side. For example, flipping the end coins changes THH into HTT. We win when all three coins are tails. There a eight possible states: HHH, HHT,...,TTT. The only *-* is HHH. The only *+* is TTT. Draw this NFA, labeling any edge that can flip between states with the letter *m*.

Convert this NFA into a regular expression. Is $m^3$ or $m^5$ in the language of this machine? The shortest word in this language is the shortest solution of this puzzle. What is it?

**Answer 7.20**

# **Chapter 8: Finite Automata with Output**

## **Functions**

In [ ]:
def display_moore(name):
    moore = Moore[name]
    header = moore[0]
    states = moore[1:]
    inputs = header[:-1]
    dot = graphviz.Digraph(name, format='png')
    dot.attr(rankdir='LR', labelloc='t')
    dot.attr('node', shape='circle')
    for i, state in enumerate(states):
        dot.node(str(i), label=f'<q<SUB>{i}</SUB>/{state[len(header) - 1]}>')
    edge_labels = {}
    for i, state in enumerate(states):
        for j, letter in enumerate(inputs):
            key = (i, state[j])
            if key not in edge_labels:
                edge_labels[key] = []
            edge_labels[key].append(letter)
    for (src, dst), letters in edge_labels.items():
        dot.edge(str(src), str(dst), label=",".join(letters))
    display(dot)

In [ ]:
def display_mealy_collapsed(name):
    mealy = Mealy[name]
    header = mealy[0]
    states = mealy[1:]
    inputs = [header[i] for i in range(0, len(header), 2)]
    dot = graphviz.Digraph(name, format='png')
    dot.attr(rankdir='LR', labelloc='t')
    dot.attr('node', shape='circle')
    for i in range(len(states)):
        dot.node(str(i), label=f'<q<SUB>{i}</SUB>>')
    edge_labels = {}
    for i, state in enumerate(states):
        for j, letter in enumerate(inputs):
            key = (i, state[2*j])
            if key not in edge_labels:
                edge_labels[key] = []
            edge_labels[key].append(f"{letter}/{state[2*j + 1]}")
    for (src, dst), labels in edge_labels.items():
        dot.edge(str(src), str(dst), label=",".join(labels))
    display(dot)

In [ ]:
def display_mealy(name):
    mealy = Mealy[name]
    header = mealy[0]
    states = mealy[1:]
    inputs = [header[i] for i in range(0, len(header), 2)]
    dot = graphviz.Digraph(name, format='png')
    dot.attr(rankdir='LR', labelloc='t')
    dot.attr('node', shape='circle')
    for i in range(len(states)):
        dot.node(str(i), label=f'<q<SUB>{i}</SUB>>')
    for i, state in enumerate(states):
        for j, letter in enumerate(inputs):
            dot.edge(str(i), str(state[2*j]), label=f"{letter}/{state[2*j + 1]}")
    display(dot)

## Problem 8.1:

---
Each of the following is a Moore machine with alphabet Σ = $\{ a\, b \}$ and output alphabet Γ = $\{ 0\, 1\}$. Given the transition and output tables, draw the machines.

**Answer 8.1:** *See below*

In [ ]:
#-------------------------------------------------------------------------------
Moore['prob8_1_1'] = (\
  [ ['a', 'b', 'Output']
  , [ 1 ,  2 ,  1 ] # 0
  , [ 1 ,  1 ,  0 ] # 1
  , [ 1 ,  0 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
Moore['prob8_1_2'] = (\
  [ ['a', 'b', 'Output']
  , [ 0 ,  2 ,  0 ] # 0
  , [ 1 ,  0 ,  1 ] # 1
  , [ 2 ,  1 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
Moore['prob8_1_3'] = (\
  [ ['a', 'b', 'Output']
  , [ 0 ,  1 ,  1 ] # 0
  , [ 0 ,  2 ,  0 ] # 1
  , [ 2 ,  2 ,  1 ] # 2
  , [ 1 ,  1 ,  0 ] # 3
])
#-------------------------------------------------------------------------------
Moore['prob8_1_4'] = (\
  [ ['a', 'b', 'Output']
  , [ 3 ,  2 ,  0 ] # 0
  , [ 1 ,  0 ,  0 ] # 1
  , [ 2 ,  3 ,  1 ] # 2
  , [ 0 ,  1 ,  0 ] # 3
])
#-------------------------------------------------------------------------------
Moore['prob8_1_5'] = (\
  [ ['a', 'b', 'Output']
  , [ 1 ,  2 ,  0 ] # 0
  , [ 2 ,  3 ,  0 ] # 1
  , [ 3 ,  4 ,  1 ] # 2
  , [ 4 ,  4 ,  0 ] # 3
  , [ 0 ,  0 ,  0 ] # 4
])
#-------------------------------------------------------------------------------
display_moore('prob8_1_1')
display_moore('prob8_1_2')
display_moore('prob8_1_3')
display_moore('prob8_1_4')
display_moore('prob8_1_5')

## Problem 8.2:

---
* (i) Based on the table representation for Moore machines, how many different *Mo*'s are there with four states?
* (ii) How many different Moore machines are there with *n* states?

**Answer 8.2:**

## Problem 8.3:

---
For each of the following Moore machines, construct the transition and output tables:

**Answer 8.3:**

In [ ]:
#-------------------------------------------------------------------------------
Moore['prob8_3_1'] = (\
  [ ['a', 'b', 'Output']
  , [ 0 ,  1 ,  0 ] # 0
  , [ 1 ,  1 ,  1 ] # 1
])
#-------------------------------------------------------------------------------
Moore['prob8_3_2'] = (\
  [ ['a', 'b', 'Output']
  , [ 1 ,  1 ,  0 ] # 0
  , [ 0 ,  0 ,  1 ] # 1
])
#-------------------------------------------------------------------------------
Moore['prob8_3_3'] = (\
  [ ['a', 'b', 'Output']
  , [ 1 ,  2 ,  0 ] # 0
  , [ 1 ,  1 ,  1 ] # 1
  , [ 1 ,  0 ,  0 ] # 2
])
#-------------------------------------------------------------------------------
Moore['prob8_3_4'] = (\
  [ ['a', 'b', 'Output']
  , [ 1 ,  1 ,  1 ] # 0
  , [ 2 ,  2 ,  0 ] # 1
  , [ 2 ,  2 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
Moore['prob8_3_5'] = (\
  [ ['a', 'b', 'Output']
  , [ 2 ,  3 ,  1 ] # 0
  , [ 3 ,  2 ,  0 ] # 1
  , [ 1 ,  2 ,  1 ] # 2
  , [ 2 ,  1 ,  1 ] # 3
])
#-------------------------------------------------------------------------------
display_moore('prob8_3_1')
display_moore('prob8_3_2')
display_moore('prob8_3_3')
display_moore('prob8_3_4')
display_moore('prob8_3_5')

## Problem 8.4:

---
On each of the Moore machines in Problems 1 and 3, run the input sequence *aabab*. What are their respective outputs?

**Answer 8.4:**

## Problem 8.5:

---
Suppose we define a Less machine to be a Moore machine that does not automatically print the character of the start state. The first character it prints is the character of the second state it enters. From then on, for every state it enters it prints a character, even when it reenters the start state. In this way, the input strings get to have some say in what the first character printed is going to be. Show that these Less machines are equivalent to Mealy machines in the direct sense, that is, for every Less machine there is a Mealy machine that has the same output for every input string.

**Answer 8.5:**

## Problem 8.6:

---
Mealy machines can also be defined by transition tables. The rows and columns are both labeled with the names of the states. The entry in the table is the label of the edge (or edges) going from the row state to the column state (if there is no such edge, this entry is blank).

Construct the transition table for each of the four Mealy machines shown below:

**Answer 8.6:**

In [ ]:
#-------------------------------------------------------------------------------
Mealy['prob8_6_1'] = (\
  [ ['a', '_', 'b', '_']
  , [ 0 ,  0 ,  1 ,  1 ] # 0
  , [ 0 ,  1 ,  1 ,  0 ] # 1
])
#-------------------------------------------------------------------------------
Mealy['prob8_6_2'] = (\
  [ ['a', '_', 'b', '_']
  , [ 2 ,  0 ,  1 ,  1 ] # 0
  , [ 0 ,  0 ,  2 ,  0 ] # 1
  , [ 0 ,  1 ,  2 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
Mealy['prob8_6_3'] = (\
  [ ['a', '_', 'b', '_']
  , [ 1 ,  0 ,  1 ,  0 ] # 0
  , [ 1 ,  0 ,  1 ,  1 ] # 1
])
#-------------------------------------------------------------------------------
Mealy['prob8_6_4'] = (\
  [ ['a', '_', 'b', '_']
  , [ 0 ,  0 ,  1 ,  1 ] # 0
  , [ 1 ,  1 ,  2 ,  0 ] # 1
  , [ 2 ,  0 ,  0 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
display_mealy('prob8_6_1')
display_mealy('prob8_6_2')
display_mealy('prob8_6_3')
display_mealy('prob8_6_4')

## Problem 8.7:

---
The example of the increment machine on p. 154 used three states to perform its job. Show that two states are all that is needed.

**Answer 8.7:**

## Problem 8.8:

---
Convert the Moore machines in Problem 3 into Mealy machines.

**Answer 8.8:**

## Problem 8.9:

---
Convert the Mealy machines in Problem 6 into Moore machines.

**Answer 8.9:**

## Problem 8.10:

---
Draw a Mealy machine equivalent to the following sequential circuit:

**Answer 8.10:**

## Problem 8.11:

---
Construct a Mealy machine that produces an output string of solid 1's no matter what the input string is.

**Answer 8.11:**

## Problem 8.12:

---
* (i) Design a machine to perform a parity check on the input string; that is, the output string ends in 1 if the total number of 1-bits in the input string is odd and 0 if the total number of 1-bits in the input string is even (the front part of the output string is ignored).
* (ii) In your answer to (i), did you choose a Mealy or Moore machine and why was that the right choice?

**Answer 8.12:**

## Problem 8.13:

---
Given a bit string of length *n*, the shift-left-cyclic operation places the first bit at the end, leaving the rest of the bits unchanged. For example, SLC (100110) = 001101.

* (i) Build a Mealy machine with input and output alphabet \{ 0 1 \$ \} such that for any bit string *x* when we input the $n+1$ bits *x\$*, we get as output the $n+1$ bit string \$$SLC(x)$.
* (ii) Explain why this cannot be done without a \$.

**Answer 8.13:**

## **For Problems 14 through 16**

---
Let $(Me)^2$ mean that given a Mealy machine, an input string is processed and then the output string is immediately fed into the machine (as input) and reprocessed. Only this second resultant output is considered the final output of $(Me)^2$. If the final output string is the same as the original input string, we say that $(Me)^2$ has an identity property. Symbolically, we write $(Me)^2$ = identity.

## Problem 8.14:

---
Let $Me_1$ be the identity Mealy machine pictured below. Let $Me_2$ be the 1's compliment Mealy machine pictured below. Prove that both $(Me_1)^2$ and $(Me_2)^2$ have the identity property that the result of processing any bit string is the original string again.

**Answer 8.14:**

In [ ]:
#-------------------------------------------------------------------------------
Mealy['prob8_14_1'] = (\
  [ ['0', '_', '1', '_']
  , [ 0 ,  0 ,  0 ,  1 ] # 0
])
#-------------------------------------------------------------------------------
Mealy['prob8_14_2'] = (\
  [ ['0', '_', '1', '_']
  , [ 0 ,  1 ,  0 ,  0 ] # 0
])
#-------------------------------------------------------------------------------
display_mealy('prob8_14_1')
display_mealy('prob8_14_2')

## Problem 8.15:

---
Show that the following machine also has this identity property:

**Answer 8.15:**

In [ ]:
#-------------------------------------------------------------------------------
Mealy['prob8_15'] = (\
  [ ['0', '_', '1', '_']
  , [ 1 ,  1 ,  1 ,  0 ] # 0
  , [ 1 ,  0 ,  1 ,  1 ] # 1
])
#-------------------------------------------------------------------------------
display_mealy('prob8_15')

## Problem 8.16:

---
Find yet another Mealy machine with this property.

**Answer 8.16:**

## **For Problems 17 and 18**

---
Similarly, given two Mealy machines, let $(Me_1)(Me_2)$ mean that an input string is processed on $Me_1$ and then the output string is immediately fed into $Me_2$ (as input) and reprocessed. Only this second resultant output is considered the final output of $(Me_1)(Me_2)$. If the final output string is the same as the original input string, we say that $(Me_1)(Me_2)$ has an identity property. Symbolically, we write $(Me_1)(Me_2)$ = identity.

Given two specific machines such that $(Me_1)(Me_2)$ reproduces the original bit string, we aim to prove (in the following two problems) that $(Me_1)(Me_2)$ must necessarily also have this property.

## Problem 8.17:

---
Show that the $2^n$ possible *n*-bit strings when fed into $Me_1$ give $2^n$ different outputs.

**Answer 8.17:**

## Problem 8.18:

---
Take the equality $(Me_1)(Me_2)$ = identity. Multiply both sides by $Me_1$ to get $(Me_1)(Me_2)(Me_1)=$ identity $(Me_1) = (Me_1)$. This means that $(Me_2)(Me_1)$ takes all outputs from $Me_1$ and leaves them unchanged. Show that this observation completes the proof.

**Answer 8.18:**

## Problem 8.19:

---
You are given these two Mealy machines pictured below. Notice that they are indeed different and show that each is the inverse machine of the other, that means that $(Me_1)(Me_2)=$ identity $=(Me_2)(Me_1)$.

**Answer 8.19:**

In [ ]:
#-------------------------------------------------------------------------------
Mealy['prob8_19_1'] = (\
  [ ['0', '_', '1', '_']
  , [ 1 ,  1 ,  2 ,  0 ] # 0
  , [ 1 ,  0 ,  1 ,  1 ] # 1
  , [ 2 ,  1 ,  2 ,  0 ] # 2
])
#-------------------------------------------------------------------------------
Mealy['prob8_19_2'] = (\
  [ ['0', '_', '1', '_']
  , [ 1 ,  1 ,  2 ,  0 ] # 0
  , [ 1 ,  1 ,  1 ,  0 ] # 1
  , [ 2 ,  0 ,  2 ,  1 ] # 2
])
#-------------------------------------------------------------------------------
display_mealy('prob8_19_1')
display_mealy('prob8_19_2')

## Problem 8.20:

---
Prove that there is no Mealy machine that reverses an input string, that is, $Me(s) =$ transpose(*s*).

**Answer 8.20:**

# **Chapter 9: Regular Languages**

## **Problems 1 through 19**

For each of the following pairs of regular languages, find a regular expression and an FA that each define $L_1 \cap L_2$:

**Problem 1:** $L_1=(a+b)^{*}a \quad L_2=b(a+b)^{*}$

**Problem 2:** $L_1=(a+b)^{*}a \quad L_2=(a+b)^{*}aa(a+b)^{*}$

**Problem 3:** $L_1=(a+b)^{*}a \quad L_2=(a+b)^{*}b$

**Problem 4:** $L_1=(a+b)b(a+b)^{*} \quad L_2=b(a+b)^{*}$

**Problem 5:** $L_1=(a+b)b(a+b)^{*} \quad L_2=(a+b)^{*}aa(a+b)^{*}$

**Problem 6:** $L_1=(a+b)b(a+b)^{*} \quad L_2=(a+b)^{*}b$

**Problem 7:** $L_1=(b+ab)^{*}(a+Λ) \quad L_2=(a+b)^{*}aa(a+b)^{*}$

**Problem 8:** $L_1=(b+ab)^{*}(a+Λ) \quad L_2=(b+ab^{*}a)^{*}ab^{*}$

**Problem 9:** $L_1=(b+ab)^{*}(a+Λ) \quad L_2=(a+ba)^{*}a$

**Problem 10:** $L_1=(ab^{*})^{*} \quad L_2=b(a+b)^{*}$

**Problem 11:** $L_1=(ab^{*})^{*} \quad L_2=a(a+b)^{*}$

**Problem 12:** $L_1=(ab^{*})^{*} \quad L_2=(a+b)^{*}aa(a+b)^{*}$

**Problem 13:** $L_1=(EVEN\_STRING) \quad L_2=b(a+b)^{*}$

**Problem 14:** $L_1=(EVEN\_STRING) \quad L_2=(a+b)^{*}aa(a+b)^{*}$

**Problem 15:** $L_1=(EVEN\_STRING) \quad L_2=(b+ab)^{*}(a+Λ)$

**Problem 16:** $L_1=(ODD\_STRING) \quad L_2=a(a+b)^{*}$

**Problem 17:** $L_1=(EVEN\_STRING) \quad L_2=EVEN\_EVEN$

**Problem 18(i):** $L_1=(EVEN\_STRING) \quad L_2=(EVEN\_A)$

**Problem 18(ii):** $L_1=(EVEN\_STRING) \quad L_2=(ODD\_A)$

**Problem 19(i):** $L_1=(EVEN\_STRING) \quad L_2=(ODD\_ODD)$

**Problem 19(ii):** $L_1=(EVEN\_STRING) \quad L_2=(ODD\_EVEN)$

## **Problem 20:**

---
We have seen that because the regular languages are closed under union and compliment, they must be closed under intersection. Find a collection of languages that is closed under union and intersection but *not* under compliment.

**Answer 20:**

# **Chapter 10: Nonregular Languages**

## **Problem 1:**

---
Use the pumping lemma to show that each of these languages is nonregular:

* (i) $\{ a^nb^{n+1}\}$
* (ii) $\{ a^nb^na^n \}$
* (iii) $\{ a^nb^{2n} \}$
* (iv) $\{ a^nba^n \}$
* (v) $\{ a^nb^na^m \}$

**Answer 1:**

## **Problem 2:**

---
Prove that the five languages in Problem 1 are nonregular using the Myhill-Nerode theorem.

**Answer 2:**

## **Problem 3:**

---
Use the pumping lemma to prove that the language DOUBLEWORD from p. 200 is nonregular.

$$DOUBLEWORD=\{ \text{all words of the form SS, where S is any string of a's and b's} \}$$

**Answer 3:**

## **Problem 4:**

---
Define the language TRAILING_COUNT as any string *s* followed by a number of *a*'s equal to length(*s*).

TRAILING_COUNT = { *aa* *ba* *aaaa* *abaa* *baaa* *bbaa* *aaaaaa* *aabaaa* *abaaaa* ... }

Prove that this language is nonregular by the
* (i) Pumping lemma.
* (ii) Myhill-Nerode theorem.

**Answer 4:**

## **Problem 5:**

---
Define the languages

$$
EVENPALINDROME = \{ \text{all words in PALINDROME that have even length} \}
$$

$$
ODDPALINDROME = \{ \text{all words in PALINDROME that have odd length} \}
$$

* (i) Show that each is nonregular by the pumping lemma.
* (ii) Show that each is nonregular by the Myhill-Nerode theorem.

**Answer 5:**

## **Problem 6:**

---
Define the language SQUARE as follows:

$$
SQUARE= \{ a^n \text{where n is a square} \}
$$

This language could also be written as $\{ a^{n^2} \}$.

* (i) Use the pumping lemma to prove that SQUARE is nonregular.
* (ii) Use the Myhill-Nerode theorem to prove that SQUARE is nonregular.

**Answer 6:**

## **Problem 7:**

---
Define the language DOUBLESQUARE as follows:

$$
DOUBLESQUARE= \{ a^nb^n\, \text{where n is a square} \}
$$

Prove that DOUBLESQUARE is nonregular by the
* (i) Pumping lemma.
* (ii) Myhill-Nerode theorem.

**Answer 7:**

## **Problem 8:**

---
Define the language DOUBLEPRIME as follows:

$$
DOUBLEPRIME= \{ a^pb^p\, \text{where p is any prime} \}
$$

Prove that DOUBLEPRIME is nonregular by the
* (i) Pumping lemma.
* (ii) Myhill-Nerode theorem.

**Answer 8:**

## **Problem 9:**

---
Define the language DOUBLEFACTORIAL as follows:

$$
DOUBLEFACTORIAL= \{ a^{n!}b^{n!} \}
$$

Prove that DOUBLEFACTORIAL is nonregular by the
* (i) Pumping lemma.
* (ii) Myhill-Nerode theorem.

**Answer 9:**

## **Problem 10:**

---
Just for this problem, let the alphabet Σ = \{ *a* *b* *c* \}. Let us consider the language $a^nb^nc^n$.

Prove that this language is nonregular by the
* (i) Pumping lemma.
* (ii) Myhill-Nerode theorem.

**Answer 10:**

## **Problem 11:**

---
Let us revisit the language DOUBLEWORD from p. 200. Use the Myhill-Nerode theorem to show that this language is nonregular by showing that all the strings in $a^{*}$ are in different classes.

**Answer 11:**

## **Problem 12:**

---
Let us consider the language of algebraic expression, ALEX, defined by the recursive definition on p. 29. We never attempted to give a regular expression for this language because it is nonregular. Prove this using the Myhill-Nerode theorem and the sequence $$(x\, ((x\, (((x\, ...$$

**Answer 12:**

## **Problem 13:**

---
Define the language MOREA as follows:

MOREA = { all strings of *a*'s and *b*'s in which the total number of *a*'s is greater than the total number of *b*'s }

* (i) Use the fact that $$MOREA' \cap MOREB' \cap (a+b)^{*} = EQUAL$$ to prove that MOREA is nonregular (where MOREB has its obvious meaning).
* (ii) Explain why the pumping lemma cannot be used to prove that MOREA is nonregular.
* (iii) Show that MOREA can be shown to be nonregular by the Myhill-Nerode theorem by using the sequence $$aab\, aaab\, aaaab\, aaaaab\, ...$$

**Answer 13:**

## **Problem 14:**

---
Let $L_1, L_, L_3, ...$ be an infinite sequence of regular languages.

* (i) Let $L$ be the infinite union of all these languages taken together. Is $L$ necessarily regular?
* (ii) Is the infinite intersection of all these languages necessarily regular?

**Answer 14:**

## **Problem 15:**

---
* (i) Give an example of a regular language *R* and a nonregular language *N* such that $R+N$ is regular.
* (ii) Give an example of a regular language *R* and a nonregular language *N* such that $R+N$ is nonregular.

**Answer 15:**

## **Problem 16:**

---
Consider the following lanugage:

$$PRIME'=\{ a^n \text{where n is not a prime \} }$$

* (i) Prove that PRIME' is nonregular.
* (ii) Prove, however, that PRIME' *does* satisfy the pumping lemma.
* (iii) How can this be?

**Answer 16:**

## **Problem 17:**

---
* (i) Show that if we add a finite set of words to a regular language, the result is a regular language.
* (ii) Show that if we subtract a finite set of words from a regular language, the result is a regular language.
* (iii) Show that if we add a finite set of words to a nonregular language, the result is a nonregular language.
* (iv) Show that if we subtract a finite set of words from a regular language, the result is a nonregular language.

**Answer 17:**

## **Problem 18:**

---
The proof of Theorem 16 used FAs to show that the language $P/Q$ is regular. Show that the language $P/Q$ is regualar using the Myhill-Nerode theorem instead.

**Answer 18:**

## **Problem 19:**

---
Let us define the language PARENTHESIS to be the set of all algebraic expressions from which everything but the parenthesis have been deleted. For example, the expression $(3+(4*7)+(8+9))+(2+1)$ becomes the word $(()())()$.

$$
PARETHESIS= \{ Λ\; ()\; (())\; ()()\; ((()))\; (())()\; ()(())\; ()()()\; ... \}
$$

* (i) Show that this language is nonregular using the Myhill-Nerode theorem.
* (ii) Show that the pumping lemma cannot be successful in proving that this language is nonregular.
* (iii) If we convert the character "(" into the letter *a* and the character ")" into the letter *b*, show that PARENTHESIS becomes a subset of the language EQUAL in which each word has the property that when read from left to right, there are never more *b*'s than *a*'s.

**Answer 19:**

## **Problem 20:**

---
Consider what happens when an FA is built for an infinite language over the one-letter alphabet Σ = { *a* }. When the input is a string of *a*'s that is longer than the number of states, the path it traces must take the form of some initial sequence of edges followed by a circuit. Because all the words in the language accepted by the machine are strings of *a*'s, all the long words accepted by this FA follow the same path up to the circuit and then around and around as in the picture below:

$$PICTURE$$

Some of the states leading up to the circuit may be final states and some of the states in the circuit may be final states. This means by placing *+* signs judiciously along a long path to the circuit, we can make the machine accept any finite set of words $S_1$. While going around the circuit the first time, the FA can accept another finite set of words $S_2$. If the length of the circuit is *n*, all words of the form $a^n$ times a word in $S_2$ will also be accepted on the second go-around of the circuit.

* (i) Prove that if $L$ is any regular language over the alphabet Σ = { *a* }, then there are two finite sets of words $S_1$ and $S_2$ and an integer *n* such that $$L=S_1+S_2(a^n)^{*}$$
* (ii) Consider the language $L$ defined as

$$=\{ a^n\; \text{where n is any integer with an even number of digits in base 10} \}$$

$$L=\{ Λ\; a^{10}\; a^{11}\; a^{12}\; ... \}$$

Prove that $L$ is nonregular.

**Answer 20:**

# **Toy Languages**

## Example Sets


---
$INTEGERS:$

Rule 1: $\quad 1 \in INTEGERS$

Rule 2: $\quad x \in INTEGERS \implies x+1 \in INTEGERS$


---
$EVEN:$

Rule 1: $\quad 2 \in EVEN$

Rule 2: $\quad x \in EVEN \implies x+2 \in EVEN$


---
$ODD:$

Rule 1: $\quad 1 \in ODD$

Rule 2: $\quad x \in ODD \implies x+2 \in ODD$


---
$RATIONAL:$

Rule 1: $\quad 1 \in RATIONAL$

Rule 2: $\quad x,y \in RATIONAL \implies x+y, x * y,$ and $x/y \in RATIONAL$


## Example Languages


### $L_1 = \{x^n \mid n \geq 1\}$

$L_1 = \{x\} \cup \{\,αx \mid α \in L_1\,\}.$

Rule 1: $\quad x \in L_1$

Rule 2: $\quad α \in L_1 \implies αx \in L_1.$

RE: $\quad x^{+}$

In [ ]:
Alphabet="x"
test_re("$x^{+}$", 7, True)

### $L_2 = \{(xx)^n \mid n \geq 1\}$

Rule 1: $\quad xx \in L_2$

Rule 2: $\quad α \in L_2 \implies αxx \in L_2$

RE: $\quad (xx)^{+}$


In [ ]:
Alphabet="x"
test_re("$(xx)^{+}$", 7, True)

### $L_3=$

---
$L_3:$

Rule 1: $\quad 1,2,3,4,5,6,7,8,9 \in L_3$

Rule 2: $\quad δ_1, δ_2 \in L_3 \implies δ_20$, $ δ_2δ_1 \in L_3$

RE: $\quad (1+2+3+4+5+6+7+8+9)(0+1+2+3+4+5+6+7+8+9)^{*}$


In [ ]:
Alphabet="0123456789"
test_re("$(1+2+3+4+5+6+7+8+9)(0+1+2+3+4+5+6+7+8+9)^{*}$", 3, True)

### $L_4 = \{x^n \mid n \geq 0\}$

Rule 1: $\quad Λ \in L_4$

Rule 2: $\quad α \in L_4 \implies αx \in L_4$

RE = $\quad x^{*}$

In [ ]:
Alphabet="x"
test_re("$x^{*}$", 7, True)

### $PALINDROME:$

Rule 1: $\quad Λ,a,b \in PALINDROME$

Rule 2: $\quad γ \in PALINDROME \implies γγ, aγa, bγb \in PALINDROME$

RE: *Not possible*

### $EVEN\_STRING:$

Rule 1: $\quad Λ \in EVEN\_STRING$

Rule 2: $\quad γ \in EVEN\_STRING \implies γaa, γab, γba, γbb \in EVEN\_STRING$

In [ ]:
Alphabet="ab"
test_re("$(aa+ab+ba+bb)^{*}$", 6, True)

### $ODD\_STRING:$

Rule 1: $\quad a,b \in ODD\_STRING$

Rule 2: $\quad γ \in ODD\_STRING \implies γaa, γab, γba, γbb \in ODD\_STRING$

In [ ]:
Alphabet="ab"
test_re("$(a+b)(aa+ab+ba+bb)^{*}$", 6, True)

### $THREE\_STRING:$

Rule 1: $\quad Λ \in THREE\_STRING$

Rule 2: $\quad γ \in THREE\_STRING \implies γaaa, γaab, γaba, γabb,  γbaa, γbab, γbba, γbbb \in THREE\_STRING$

In [ ]:
Alphabet="ab"
test_re("$((a+b)(a+b)(a+b))^{*}$", 6, True)
test_re("$(aaa+aab+aba+abb+baa+bab+bba+bbb)^{*}$", 6, True)

### $MODN\_STRING:$

Rule 1: $\quad Λ \in MODN\_STRING$

Rule 2: $\quad γ \in MODN\_STRING \implies γΣ^n \in MODN\_STRING \mid Σ = \{a,b\}$

RE: $\quad ((a+b)^{n})^{*}$

### $SAME\_SAME:$

Rule 1: $\quad Λ \in SAME\_SAME$

Rule 2: $\quad γ \in SAME\_SAME \implies γab, aγb, γba, bγa \in SAME\_SAME$

### $EVEN\_EVEN:$

Rule 1: $\quad Λ \in EVEN\_EVEN$

Rule 2: $\quad γ \in EVEN\_EVEN \implies γaa, aγa, aaγ, γbb, bγb, bbγ \in EVEN\_EVEN$

In [ ]:
Alphabet="ab"
EVEN_EVEN="(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}"
test_re(f"${EVEN_EVEN}$", 6, True)

### $ODD\_ODD:$

Rule 1: $\quad ab,ba \in ODD\_ODD$

Rule 2: $\quad γ \in ODD\_ODD \implies γaa, aγa, aaγ, γbb, bγb, bbγ \in ODD\_ODD$

In [ ]:
test_re(f"${EVEN_EVEN}" + "(ab+ba)" + f"{EVEN_EVEN}$", 6, True)

### $EVEN\_ODD:$

Rule 1: $\quad b \in EVEN\_ODD$

Rule 2: $\quad γ \in EVEN\_ODD \implies γaa, aγa, aaγ, γbb, bγb, bbγ \in EVEN\_ODD$

In [ ]:
test_re(f"${EVEN_EVEN}" + "(b+ab(bb)^{*}a)" + f"{EVEN_EVEN}$", 5, True)

### $ODD\_EVEN:$

Rule 1: $\quad a \in ODD\_EVEN$

Rule 2: $\quad γ \in ODD\_EVEN \implies γaa, aγa, aaγ, γbb, bγb, bbγ \in ODD\_EVEN$

RE: $\quad odd\_even = (a(aa)^{*}+bb+ba(bb+aa)^{*}b)^{+}$

In [ ]:
test_re(f"${EVEN_EVEN}" + "(a+ba(aa)^{*}b)" + f"{EVEN_EVEN}$", 5, True)

### $EVEN\_A:$

Rule 1: $\quad Λ \in EVEN\_A$

Rule 2: $\quad γ \in EVEN\_A \implies γaa, aγa, aaγ, γb, bγ \in EVEN\_A$

In [ ]:
test_re("$(b+aa+ab(b+aa)^{*}a)^{*}$", 6, True)

### $ODD\_A:$

Rule 1: $\quad a \in ODD\_A$

Rule 2: $\quad γ \in ODD\_A \implies γaa, aγa, aaγ, γb, bγ \in ODD\_A$

RE: $\quad odd\_a = $

In [ ]:
test_re("$(b+aa+ab(b+aa)^{*}a)^{*}a(b+aa+ab(b+aa)^{*}a)^{*}$", 6, True)

### $EVEN\_B:$

Rule 1: $\quad Λ \in EVEN\_B$

Rule 2: $\quad γ \in EVEN\_B \implies γbb, bγb, bbγ, γa, aγ \in EVEN\_B$

In [ ]:
test_re("$(a+bb+ba(bb+a)^{*}b)^{*}$", 6, True)

### $ODD\_B:$

Rule 1: $\quad Λ \in ODD\_B$

Rule 2: $\quad γ \in ODD\_B \implies γbb, bγb, bbγ, γa, aγ \in ODD\_B$

In [ ]:
test_re("$(a+bb+ba(bb+a)^{*}b)^{*}b(a+bb+ba(bb+a)^{*}b)^{*}$", 6, True)

### $NO\_AB:$

Rule 1: $\quad Λ \in NO\_AB$

Rule 2: $\quad γ \in NO\_AB \implies bγ, γa \in NO\_AB$

In [ ]:
test_re("$b^{*}a^{*}$", 6, True)

### $NO\_AA:$

Rule 1: $\quad Λ,a \in NO\_AA$

Rule 2: $\quad γ \in NO\_AA \implies γb, γba \in NO\_AA$

In [ ]:
test_re("$(Λ+a)(b+ba)^{*}$", 6, True)

### $NO\_AAA:$

Rule 1: $\quad Λ,a,aa \in NO\_AAA$

Rule 2: $\quad γ \in NO\_AAA \implies γb, γba, γbaa \in NO\_AAA$

In [ ]:
test_re("$(Λ+a+aa)(b+ba+baa)^{*}$", 6, True)

### $BEGIN\_AA:$

Rule 1: $\quad aa \in BEGIN\_AA$

Rule 2: $\quad γ \in BEGIN\_AA \implies γa, γb \in BEGIN\_AA$

In [ ]:
test_re("$aa(a+b)^{*}$", 6, True)

### $END\_AA:$

Rule 1: $\quad aa \in END\_AA$

Rule 2: $\quad γ \in END\_AA \implies aγ, bγ \in END\_AA$

In [ ]:
test_re("$(a+b)^{*}aa$", 6, True)

### $HAS\_AA:$

Rule 1: $\quad aa \in HAS\_AA$

Rule 2: $\quad γ \in HAS\_AA \implies γa, γb, aγ, bγ \in HAS\_AA$

In [ ]:
test_re("$(a+b)^{*}aa(a+b)^{*}$", 6, True)

### $PREFIX\_A\_GE\_B:$

Rule 1: $\quad Λ \in PREFIX\_A\_GE\_B$

Rule 2: $\quad γ \in PREFIX\_A\_GE\_B \implies aγ, γba \in PREFIX\_A\_GE\_B$

RE: $\quad ?$

### $ALT1:$

Rule 1: $\quad a \in ALT1$

Rule 2: $\quad γ \in ALT1 \implies γba \in ALT1$

In [ ]:
test_re("$a(ba)^{*}$", 5, True)

### $ALT2:$

Rule 1: $\quad Λ \in ALT2$

Rule 2: $\quad γ \in ALT2 \implies γba \in ALT2$

In [ ]:
test_re("$(ba)^{*}$", 5, True)

### $END\_SAME:$

Rule 1: $\quad ?$

Rule 2: $\quad ?$

In [ ]:
test_re("$a(a+b)^{*}a+b(a+b)^{*}b$", 6, True)

### $BALANCED:$

Rule 1: $\quad a,b,c,...,z \in BALANCED$

Rule 2: $\quad γ \in BALANCED \implies (γ), γ^{+} \in BALANCED$

RE: *Not Possible*

### $POLY:$

* $x \in POLY, INTEGERS \subset POLY$
* $x_1 \in POLY \implies (x_1) \in POLY$
* $x_1,x_2 \in POLY \implies x_1x_2 \in POLY$
* $x_1,x_2 \in POLY \implies x_1-x_2 \in POLY$
* $x_1,x_2 \in POLY \implies x_1+x_2 \in POLY$

RE: *Not Possible*

### $AE:$

* $INTEGERS \subset AE$
* $α \in AE \implies (α) \in AE$
* $α \in AE \implies -α \in AE$
* $α,β \in AE \implies α$ ** $β \in AE$
* $α,β \in AE \implies α / β \in AE$
* $α,β \in AE \implies α * β \in AE$
* $α,β \in AE \implies α - β \in AE$
* $α,β \in AE \implies α + β \in AE$

RE: *Not Possible*

### $WFF:$

* $a,b,c,...,z \in WFF$
* $ζ \in WFF \implies (ζ) \in WFF$
* $ζ \in WFF \implies ¬ζ \in WFF$
* $ζ,ξ \in WFF \implies ζ\vee ξ$
* $ζ,ξ \in WFF \implies ζ\wedge ξ$
* $ζ,ξ \in WFF \implies ζ→ξ$

RE: *Not Possible*

### $L:$

#